<a href="https://colab.research.google.com/github/1kaiser/test2022/blob/main/NSIDC_Data_Download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!wget https://lpdaac.usgs.gov/documents/762/Daac2Disk_ubuntu18

In [ ]:
!sudo apt-get install /content/Daac2Disk_ubuntu18

In [ ]:
!sudo /content/Daac2Disk_ubuntu18 -version

hdf data in python

# MOD09A1.006

In [12]:
#@title creating the download.sh file in the folder { vertical-output: true }
%%writefile download.sh
#!/bin/bash

GREP_OPTIONS=''

cookiejar=$(mktemp cookies.XXXXXXXXXX)
netrc=$(mktemp netrc.XXXXXXXXXX)
chmod 0600 "$cookiejar" "$netrc"
function finish {
  rm -rf "$cookiejar" "$netrc"
}

trap finish EXIT
WGETRC="$wgetrc"

prompt_credentials() {
    echo "Enter your Earthdata Login or other provider supplied credentials"
    
    username=kroy0001
    password=/#j%kWrPA,8.HRe
    echo "machine urs.earthdata.nasa.gov login $username password $password" >> $netrc
    echo
}

exit_with_error() {
    echo
    echo "Unable to Retrieve Data"
    echo
    echo $1
    echo
    echo "https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2000.02.18/MOD09A1.A2000049.h24v05.006.2015136143410.hdf"
    echo
    exit 1
}

prompt_credentials
  detect_app_approval() {
    approved=`curl -s -b "$cookiejar" -c "$cookiejar" -L --max-redirs 5 --netrc-file "$netrc" https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2000.02.18/MOD09A1.A2000049.h24v05.006.2015136143410.hdf -w %{http_code} | tail  -1`
    if [ "$approved" -ne "302" ]; then
        # User didn't approve the app. Direct users to approve the app in URS
        exit_with_error "Please ensure that you have authorized the remote application by visiting the link below "
    fi
}

setup_auth_curl() {
    # Firstly, check if it require URS authentication
    status=$(curl -s -z "$(date)" -w %{http_code} https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2000.02.18/MOD09A1.A2000049.h24v05.006.2015136143410.hdf | tail -1)
    if [[ "$status" -ne "200" && "$status" -ne "304" ]]; then
        # URS authentication is required. Now further check if the application/remote service is approved.
        detect_app_approval
    fi
}

setup_auth_wget() {
    # The safest way to auth via curl is netrc. Note: there's no checking or feedback
    # if login is unsuccessful
    touch ~/.netrc
    chmod 0600 ~/.netrc
    credentials=$(grep 'machine urs.earthdata.nasa.gov' ~/.netrc)
    if [ -z "$credentials" ]; then
        cat "$netrc" >> ~/.netrc
    fi
}

fetch_urls() {
  if command -v curl >/dev/null 2>&1; then
      setup_auth_curl
      while read -r line; do
        # Get everything after the last '/'
        filename="${line##*/}"

        # Strip everything after '?'
        stripped_query_params="${filename%%\?*}"

        curl -f -b "$cookiejar" -c "$cookiejar" -L --netrc-file "$netrc" -g -o $stripped_query_params -- $line && echo || exit_with_error "Command failed with error. Please retrieve the data manually."
      done;
  elif command -v wget >/dev/null 2>&1; then
      # We can't use wget to poke provider server to get info whether or not URS was integrated without download at least one of the files.
      echo
      echo "WARNING: Can't find curl, use wget instead."
      echo "WARNING: Script may not correctly identify Earthdata Login integrations."
      echo
      setup_auth_wget
      while read -r line; do
        # Get everything after the last '/'
        filename="${line##*/}"

        # Strip everything after '?'
        stripped_query_params="${filename%%\?*}"

        wget --load-cookies "$cookiejar" --save-cookies "$cookiejar" --output-document $stripped_query_params --keep-session-cookies -- $line && echo || exit_with_error "Command failed with error. Please retrieve the data manually."
      done;
  else
      exit_with_error "Error: Could not find a command-line downloader.  Please install curl or wget"
  fi
}

fetch_urls <<'EDSCEOF'
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2000.02.18/MOD09A1.A2000049.h24v05.006.2015136143410.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2000.02.26/MOD09A1.A2000057.h24v05.006.2015136061408.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2000.03.05/MOD09A1.A2000065.h24v05.006.2015136061510.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2000.03.13/MOD09A1.A2000073.h24v05.006.2015136061836.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2000.03.21/MOD09A1.A2000081.h24v05.006.2015136080103.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2000.03.29/MOD09A1.A2000089.h24v05.006.2015136063726.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2000.04.06/MOD09A1.A2000097.h24v05.006.2015136080202.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2000.04.14/MOD09A1.A2000105.h24v05.006.2015136064122.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2000.04.22/MOD09A1.A2000113.h24v05.006.2015136200818.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2000.04.30/MOD09A1.A2000121.h24v05.006.2015137071757.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2000.05.08/MOD09A1.A2000129.h24v05.006.2015137091932.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2000.05.16/MOD09A1.A2000137.h24v05.006.2015137072522.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2000.05.24/MOD09A1.A2000145.h24v05.006.2015137134441.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2000.06.01/MOD09A1.A2000153.h24v05.006.2015137093517.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2000.06.09/MOD09A1.A2000161.h24v05.006.2015137072110.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2000.06.17/MOD09A1.A2000169.h24v05.006.2015137083547.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2000.06.25/MOD09A1.A2000177.h24v05.006.2015137220828.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2000.07.03/MOD09A1.A2000185.h24v05.006.2015138104809.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2000.07.11/MOD09A1.A2000193.h24v05.006.2015138105443.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2000.07.19/MOD09A1.A2000201.h24v05.006.2015138113339.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2000.07.27/MOD09A1.A2000209.h24v05.006.2015138113037.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2000.08.04/MOD09A1.A2000217.h24v05.006.2015138053036.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2000.08.12/MOD09A1.A2000225.h24v05.006.2015138073446.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2000.08.20/MOD09A1.A2000233.h24v05.006.2015138112641.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2000.08.28/MOD09A1.A2000241.h24v05.006.2015138112447.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2000.09.05/MOD09A1.A2000249.h24v05.006.2015138113116.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2000.09.13/MOD09A1.A2000257.h24v05.006.2015138234502.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2000.09.21/MOD09A1.A2000265.h24v05.006.2015139113537.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2000.09.29/MOD09A1.A2000273.h24v05.006.2015139114850.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2000.10.07/MOD09A1.A2000281.h24v05.006.2015139113834.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2000.10.15/MOD09A1.A2000289.h24v05.006.2015139115442.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2000.10.23/MOD09A1.A2000297.h24v05.006.2015139123606.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2000.10.31/MOD09A1.A2000305.h24v05.006.2015139115000.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2000.11.08/MOD09A1.A2000313.h24v05.006.2015139114234.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2000.11.16/MOD09A1.A2000321.h24v05.006.2015139115655.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2000.11.24/MOD09A1.A2000329.h24v05.006.2015140020147.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2000.12.02/MOD09A1.A2000337.h24v05.006.2015140122841.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2000.12.10/MOD09A1.A2000345.h24v05.006.2015140130345.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2000.12.18/MOD09A1.A2000353.h24v05.006.2015140123345.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2000.12.26/MOD09A1.A2000361.h24v05.006.2015140115448.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2001.01.01/MOD09A1.A2001001.h24v05.006.2015140115351.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2001.01.09/MOD09A1.A2001009.h24v05.006.2015140121407.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2001.01.17/MOD09A1.A2001017.h24v05.006.2015140120824.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2001.01.25/MOD09A1.A2001025.h24v05.006.2015140130854.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2001.02.02/MOD09A1.A2001033.h24v05.006.2015140130936.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2001.02.10/MOD09A1.A2001041.h24v05.006.2015141191823.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2001.02.18/MOD09A1.A2001049.h24v05.006.2015142184400.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2001.02.26/MOD09A1.A2001057.h24v05.006.2015142200339.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2001.03.06/MOD09A1.A2001065.h24v05.006.2015142213545.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2001.03.14/MOD09A1.A2001073.h24v05.006.2015142212454.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2001.03.22/MOD09A1.A2001081.h24v05.006.2015142212456.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2001.03.30/MOD09A1.A2001089.h24v05.006.2015142230216.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2001.04.07/MOD09A1.A2001097.h24v05.006.2015142232631.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2001.04.15/MOD09A1.A2001105.h24v05.006.2015142232626.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2001.04.23/MOD09A1.A2001113.h24v05.006.2015142234052.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2001.05.01/MOD09A1.A2001121.h24v05.006.2015142083354.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2001.05.09/MOD09A1.A2001129.h24v05.006.2015142103816.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2001.05.17/MOD09A1.A2001137.h24v05.006.2015142101517.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2001.05.25/MOD09A1.A2001145.h24v05.006.2015143040731.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2001.06.02/MOD09A1.A2001153.h24v05.006.2015143140103.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2001.06.10/MOD09A1.A2001161.h24v05.006.2015143110004.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2001.07.04/MOD09A1.A2001185.h24v05.006.2015143132749.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2001.07.12/MOD09A1.A2001193.h24v05.006.2015143162612.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2001.07.20/MOD09A1.A2001201.h24v05.006.2015143134653.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2001.07.28/MOD09A1.A2001209.h24v05.006.2015143132905.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2001.08.05/MOD09A1.A2001217.h24v05.006.2015143202505.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2001.08.13/MOD09A1.A2001225.h24v05.006.2015144090751.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2001.08.21/MOD09A1.A2001233.h24v05.006.2015144102042.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2001.08.29/MOD09A1.A2001241.h24v05.006.2015144101610.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2001.09.06/MOD09A1.A2001249.h24v05.006.2015144081857.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2001.09.14/MOD09A1.A2001257.h24v05.006.2015144103118.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2001.09.22/MOD09A1.A2001265.h24v05.006.2015144093705.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2001.09.30/MOD09A1.A2001273.h24v05.006.2015144100134.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2001.10.08/MOD09A1.A2001281.h24v05.006.2015144103222.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2001.10.16/MOD09A1.A2001289.h24v05.006.2015145002153.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2001.10.24/MOD09A1.A2001297.h24v05.006.2015146031306.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2001.11.01/MOD09A1.A2001305.h24v05.006.2015145220533.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2001.11.09/MOD09A1.A2001313.h24v05.006.2015146042414.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2001.11.17/MOD09A1.A2001321.h24v05.006.2015146055141.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2001.11.25/MOD09A1.A2001329.h24v05.006.2015146072529.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2001.12.03/MOD09A1.A2001337.h24v05.006.2015146062957.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2001.12.11/MOD09A1.A2001345.h24v05.006.2015146072745.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2001.12.19/MOD09A1.A2001353.h24v05.006.2015146072150.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2001.12.27/MOD09A1.A2001361.h24v05.006.2015146122906.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2002.01.01/MOD09A1.A2002001.h24v05.006.2015146184856.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2002.01.09/MOD09A1.A2002009.h24v05.006.2015146151212.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2002.01.17/MOD09A1.A2002017.h24v05.006.2015146144420.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2002.01.25/MOD09A1.A2002025.h24v05.006.2015146161606.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2002.02.02/MOD09A1.A2002033.h24v05.006.2015146151700.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2002.02.10/MOD09A1.A2002041.h24v05.006.2015146184448.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2002.02.18/MOD09A1.A2002049.h24v05.006.2015146163100.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2002.02.26/MOD09A1.A2002057.h24v05.006.2015146163740.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2002.03.06/MOD09A1.A2002065.h24v05.006.2015147001005.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2002.03.14/MOD09A1.A2002073.h24v05.006.2015147152016.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2002.03.22/MOD09A1.A2002081.h24v05.006.2015147030821.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2002.03.30/MOD09A1.A2002089.h24v05.006.2015147165426.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2002.04.07/MOD09A1.A2002097.h24v05.006.2015147202711.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2002.04.15/MOD09A1.A2002105.h24v05.006.2015148002907.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2002.04.23/MOD09A1.A2002113.h24v05.006.2015148003614.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2002.05.01/MOD09A1.A2002121.h24v05.006.2015148010837.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2002.05.09/MOD09A1.A2002129.h24v05.006.2015148023236.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2002.05.17/MOD09A1.A2002137.h24v05.006.2015148031646.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2002.05.25/MOD09A1.A2002145.h24v05.006.2015148125135.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2002.06.02/MOD09A1.A2002153.h24v05.006.2015148140804.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2002.06.10/MOD09A1.A2002161.h24v05.006.2015148124829.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2002.06.18/MOD09A1.A2002169.h24v05.006.2015148132042.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2002.06.26/MOD09A1.A2002177.h24v05.006.2015148190727.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2002.07.04/MOD09A1.A2002185.h24v05.006.2015149035346.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2002.07.12/MOD09A1.A2002193.h24v05.006.2015149031334.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2002.07.20/MOD09A1.A2002201.h24v05.006.2015149063259.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2002.07.28/MOD09A1.A2002209.h24v05.006.2015149054831.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2002.08.05/MOD09A1.A2002217.h24v05.006.2015149204657.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2002.08.13/MOD09A1.A2002225.h24v05.006.2015150063100.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2002.08.21/MOD09A1.A2002233.h24v05.006.2015150032811.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2002.08.29/MOD09A1.A2002241.h24v05.006.2015150052145.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2002.09.06/MOD09A1.A2002249.h24v05.006.2015151001122.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2002.09.14/MOD09A1.A2002257.h24v05.006.2015151120956.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2002.09.22/MOD09A1.A2002265.h24v05.006.2015151112219.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2002.09.30/MOD09A1.A2002273.h24v05.006.2015151113218.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2002.10.08/MOD09A1.A2002281.h24v05.006.2015151113052.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2002.10.16/MOD09A1.A2002289.h24v05.006.2015151204721.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2002.10.24/MOD09A1.A2002297.h24v05.006.2015152070809.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2002.11.01/MOD09A1.A2002305.h24v05.006.2015152051430.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2002.11.09/MOD09A1.A2002313.h24v05.006.2015152074507.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2002.11.17/MOD09A1.A2002321.h24v05.006.2015152075630.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2002.11.25/MOD09A1.A2002329.h24v05.006.2015152215151.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2002.12.03/MOD09A1.A2002337.h24v05.006.2015153085418.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2002.12.11/MOD09A1.A2002345.h24v05.006.2015153084925.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2002.12.19/MOD09A1.A2002353.h24v05.006.2015153054624.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2002.12.27/MOD09A1.A2002361.h24v05.006.2015153054818.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2003.01.01/MOD09A1.A2003001.h24v05.006.2015153103157.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2003.01.09/MOD09A1.A2003009.h24v05.006.2015153235828.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2003.01.17/MOD09A1.A2003017.h24v05.006.2015154085239.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2003.01.25/MOD09A1.A2003025.h24v05.006.2015154084142.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2003.02.02/MOD09A1.A2003033.h24v05.006.2015154101252.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2003.02.10/MOD09A1.A2003041.h24v05.006.2015156070101.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2003.02.18/MOD09A1.A2003049.h24v05.006.2015156064928.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2003.02.26/MOD09A1.A2003057.h24v05.006.2015156120826.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2003.03.06/MOD09A1.A2003065.h24v05.006.2015156125921.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2003.03.14/MOD09A1.A2003073.h24v05.006.2015156225608.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2003.03.22/MOD09A1.A2003081.h24v05.006.2015156230242.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2003.03.30/MOD09A1.A2003089.h24v05.006.2015156152831.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2003.04.07/MOD09A1.A2003097.h24v05.006.2015157035933.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2003.04.15/MOD09A1.A2003105.h24v05.006.2015157043147.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2003.04.23/MOD09A1.A2003113.h24v05.006.2015157102900.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2003.05.01/MOD09A1.A2003121.h24v05.006.2015157082010.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2003.05.09/MOD09A1.A2003129.h24v05.006.2015158183657.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2003.05.17/MOD09A1.A2003137.h24v05.006.2015158190704.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2003.05.25/MOD09A1.A2003145.h24v05.006.2015158191854.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2003.06.02/MOD09A1.A2003153.h24v05.006.2015158193201.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2003.06.10/MOD09A1.A2003161.h24v05.006.2015158192011.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2003.06.18/MOD09A1.A2003169.h24v05.006.2015159181501.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2003.06.26/MOD09A1.A2003177.h24v05.006.2015159181723.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2003.07.04/MOD09A1.A2003185.h24v05.006.2015161171609.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2003.07.12/MOD09A1.A2003193.h24v05.006.2015076090554.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2003.07.20/MOD09A1.A2003201.h24v05.006.2015076172901.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2003.07.28/MOD09A1.A2003209.h24v05.006.2015077014405.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2003.08.05/MOD09A1.A2003217.h24v05.006.2015078183125.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2003.08.13/MOD09A1.A2003225.h24v05.006.2015079061105.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2003.08.21/MOD09A1.A2003233.h24v05.006.2015079052857.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2003.08.29/MOD09A1.A2003241.h24v05.006.2015079043520.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2003.09.06/MOD09A1.A2003249.h24v05.006.2015080014202.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2003.09.14/MOD09A1.A2003257.h24v05.006.2015080042124.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2003.09.22/MOD09A1.A2003265.h24v05.006.2015080214307.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2003.09.30/MOD09A1.A2003273.h24v05.006.2015081070110.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2003.10.08/MOD09A1.A2003281.h24v05.006.2015081065516.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2003.10.16/MOD09A1.A2003289.h24v05.006.2015081072341.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2003.10.24/MOD09A1.A2003297.h24v05.006.2015082060242.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2003.11.01/MOD09A1.A2003305.h24v05.006.2015082060154.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2003.11.09/MOD09A1.A2003313.h24v05.006.2015082175809.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2003.11.17/MOD09A1.A2003321.h24v05.006.2015083062847.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2003.11.25/MOD09A1.A2003329.h24v05.006.2015083044531.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2003.12.03/MOD09A1.A2003337.h24v05.006.2015083171830.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2003.12.11/MOD09A1.A2003345.h24v05.006.2015083053549.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2003.12.19/MOD09A1.A2003353.h24v05.006.2015084004615.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2003.12.27/MOD09A1.A2003361.h24v05.006.2015084202318.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2004.01.01/MOD09A1.A2004001.h24v05.006.2015085011032.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2004.01.09/MOD09A1.A2004009.h24v05.006.2015085071455.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2004.01.17/MOD09A1.A2004017.h24v05.006.2015085081125.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2004.01.25/MOD09A1.A2004025.h24v05.006.2015112185239.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2004.02.02/MOD09A1.A2004033.h24v05.006.2015085192034.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2004.02.10/MOD09A1.A2004041.h24v05.006.2015086043443.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2004.02.18/MOD09A1.A2004049.h24v05.006.2015086041345.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2004.02.26/MOD09A1.A2004057.h24v05.006.2015086184908.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2004.03.05/MOD09A1.A2004065.h24v05.006.2015087070610.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2004.03.13/MOD09A1.A2004073.h24v05.006.2015087074141.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2004.03.21/MOD09A1.A2004081.h24v05.006.2015087193705.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2004.03.29/MOD09A1.A2004089.h24v05.006.2015088054700.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2004.04.06/MOD09A1.A2004097.h24v05.006.2015088100811.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2004.04.14/MOD09A1.A2004105.h24v05.006.2015088071404.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2004.04.22/MOD09A1.A2004113.h24v05.006.2015089195126.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2004.04.30/MOD09A1.A2004121.h24v05.006.2015089200124.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2004.05.08/MOD09A1.A2004129.h24v05.006.2015089075945.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2004.05.16/MOD09A1.A2004137.h24v05.006.2015089183923.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2004.05.24/MOD09A1.A2004145.h24v05.006.2015090055010.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2004.06.01/MOD09A1.A2004153.h24v05.006.2015090071148.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2004.06.09/MOD09A1.A2004161.h24v05.006.2015090055019.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2004.06.17/MOD09A1.A2004169.h24v05.006.2015090175718.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2004.06.25/MOD09A1.A2004177.h24v05.006.2015091061516.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2004.07.03/MOD09A1.A2004185.h24v05.006.2015091084053.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2004.07.11/MOD09A1.A2004193.h24v05.006.2015091195318.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2004.07.19/MOD09A1.A2004201.h24v05.006.2015092103321.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2004.07.27/MOD09A1.A2004209.h24v05.006.2015092092047.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2004.08.04/MOD09A1.A2004217.h24v05.006.2015092110806.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2004.08.12/MOD09A1.A2004225.h24v05.006.2015092192615.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2004.08.20/MOD09A1.A2004233.h24v05.006.2015094003133.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2004.08.28/MOD09A1.A2004241.h24v05.006.2015093084901.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2004.09.05/MOD09A1.A2004249.h24v05.006.2015093211442.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2004.09.13/MOD09A1.A2004257.h24v05.006.2015094090333.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2004.09.21/MOD09A1.A2004265.h24v05.006.2015094104120.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2004.09.29/MOD09A1.A2004273.h24v05.006.2015094102028.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2004.10.07/MOD09A1.A2004281.h24v05.006.2015094203656.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2004.10.15/MOD09A1.A2004289.h24v05.006.2015095084832.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2004.10.23/MOD09A1.A2004297.h24v05.006.2015121164423.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2004.10.31/MOD09A1.A2004305.h24v05.006.2015095194952.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2004.11.08/MOD09A1.A2004313.h24v05.006.2015096054318.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2004.11.16/MOD09A1.A2004321.h24v05.006.2015096062939.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2004.11.24/MOD09A1.A2004329.h24v05.006.2015096081724.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2004.12.02/MOD09A1.A2004337.h24v05.006.2015096210311.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2004.12.10/MOD09A1.A2004345.h24v05.006.2015097050446.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2004.12.18/MOD09A1.A2004353.h24v05.006.2015097050719.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2004.12.26/MOD09A1.A2004361.h24v05.006.2015099003711.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2005.01.01/MOD09A1.A2005001.h24v05.006.2015099032819.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2005.01.09/MOD09A1.A2005009.h24v05.006.2015099032521.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2005.01.17/MOD09A1.A2005017.h24v05.006.2015099032633.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2005.01.25/MOD09A1.A2005025.h24v05.006.2015099201946.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2005.02.02/MOD09A1.A2005033.h24v05.006.2015100070638.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2005.02.10/MOD09A1.A2005041.h24v05.006.2015100071316.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2005.02.18/MOD09A1.A2005049.h24v05.006.2015100213802.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2005.02.26/MOD09A1.A2005057.h24v05.006.2015101082630.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2005.03.06/MOD09A1.A2005065.h24v05.006.2015101102414.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2005.03.14/MOD09A1.A2005073.h24v05.006.2015101094034.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2005.03.22/MOD09A1.A2005081.h24v05.006.2015101223851.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2005.03.30/MOD09A1.A2005089.h24v05.006.2015102192906.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2005.04.07/MOD09A1.A2005097.h24v05.006.2015102195554.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2005.04.15/MOD09A1.A2005105.h24v05.006.2015103012411.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2005.04.23/MOD09A1.A2005113.h24v05.006.2015103093945.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2005.05.01/MOD09A1.A2005121.h24v05.006.2015103132424.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2005.05.09/MOD09A1.A2005129.h24v05.006.2015103130211.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2005.05.17/MOD09A1.A2005137.h24v05.006.2015103224240.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2005.05.25/MOD09A1.A2005145.h24v05.006.2015104073854.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2005.06.02/MOD09A1.A2005153.h24v05.006.2015104093809.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2005.06.10/MOD09A1.A2005161.h24v05.006.2015104200113.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2005.06.18/MOD09A1.A2005169.h24v05.006.2015105071306.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2005.06.26/MOD09A1.A2005177.h24v05.006.2015105074148.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2005.07.04/MOD09A1.A2005185.h24v05.006.2015105190517.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2005.07.12/MOD09A1.A2005193.h24v05.006.2015106045356.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2005.07.20/MOD09A1.A2005201.h24v05.006.2015106085722.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2005.07.28/MOD09A1.A2005209.h24v05.006.2015106103200.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2005.08.05/MOD09A1.A2005217.h24v05.006.2015106185642.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2005.08.13/MOD09A1.A2005225.h24v05.006.2015107053903.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2005.08.21/MOD09A1.A2005233.h24v05.006.2015107085957.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2005.08.29/MOD09A1.A2005241.h24v05.006.2015107204455.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2005.09.06/MOD09A1.A2005249.h24v05.006.2015108042156.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2005.09.14/MOD09A1.A2005257.h24v05.006.2015108073800.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2005.09.22/MOD09A1.A2005265.h24v05.006.2015108063017.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2005.09.30/MOD09A1.A2005273.h24v05.006.2015109035402.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2005.10.08/MOD09A1.A2005281.h24v05.006.2015109144812.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2005.10.16/MOD09A1.A2005289.h24v05.006.2015109160753.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2005.10.24/MOD09A1.A2005297.h24v05.006.2015109232451.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2005.11.01/MOD09A1.A2005305.h24v05.006.2015110074046.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2005.11.09/MOD09A1.A2005313.h24v05.006.2015110094317.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2005.11.17/MOD09A1.A2005321.h24v05.006.2015110094931.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2005.11.25/MOD09A1.A2005329.h24v05.006.2015110210855.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2005.12.03/MOD09A1.A2005337.h24v05.006.2015112060634.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2005.12.11/MOD09A1.A2005345.h24v05.006.2015112045434.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2005.12.19/MOD09A1.A2005353.h24v05.006.2015112054327.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2005.12.27/MOD09A1.A2005361.h24v05.006.2015112231534.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2006.01.01/MOD09A1.A2006001.h24v05.006.2015113050645.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2006.01.09/MOD09A1.A2006009.h24v05.006.2015113075327.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2006.01.17/MOD09A1.A2006017.h24v05.006.2015113070347.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2006.01.25/MOD09A1.A2006025.h24v05.006.2015113044834.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2006.02.02/MOD09A1.A2006033.h24v05.006.2015114055739.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2006.02.10/MOD09A1.A2006041.h24v05.006.2015114055934.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2006.02.18/MOD09A1.A2006049.h24v05.006.2015114064421.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2006.02.26/MOD09A1.A2006057.h24v05.006.2015114065256.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2006.03.06/MOD09A1.A2006065.h24v05.006.2015115042453.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2006.03.14/MOD09A1.A2006073.h24v05.006.2015115050429.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2006.03.22/MOD09A1.A2006081.h24v05.006.2015115054310.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2006.03.30/MOD09A1.A2006089.h24v05.006.2015115053825.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2006.04.07/MOD09A1.A2006097.h24v05.006.2015116080028.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2006.04.15/MOD09A1.A2006105.h24v05.006.2015116073701.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2006.04.23/MOD09A1.A2006113.h24v05.006.2015116081100.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2006.05.01/MOD09A1.A2006121.h24v05.006.2015116082626.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2006.05.09/MOD09A1.A2006129.h24v05.006.2015117051148.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2006.05.17/MOD09A1.A2006137.h24v05.006.2015117042133.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2006.05.25/MOD09A1.A2006145.h24v05.006.2015117053018.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2006.06.02/MOD09A1.A2006153.h24v05.006.2015117054925.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2006.06.10/MOD09A1.A2006161.h24v05.006.2015119064456.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2006.06.18/MOD09A1.A2006169.h24v05.006.2015119045949.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2006.06.26/MOD09A1.A2006177.h24v05.006.2015119061609.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2006.07.04/MOD09A1.A2006185.h24v05.006.2015119072548.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2006.07.12/MOD09A1.A2006193.h24v05.006.2015120063433.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2006.07.20/MOD09A1.A2006201.h24v05.006.2015120060701.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2006.07.28/MOD09A1.A2006209.h24v05.006.2015120070615.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2006.08.05/MOD09A1.A2006217.h24v05.006.2015120061421.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2006.08.13/MOD09A1.A2006225.h24v05.006.2015121065248.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2006.08.21/MOD09A1.A2006233.h24v05.006.2015121075253.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2006.08.29/MOD09A1.A2006241.h24v05.006.2015121075922.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2006.09.06/MOD09A1.A2006249.h24v05.006.2015121092811.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2006.09.14/MOD09A1.A2006257.h24v05.006.2015122232937.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2006.09.22/MOD09A1.A2006265.h24v05.006.2015123105217.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2006.09.30/MOD09A1.A2006273.h24v05.006.2015123111959.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2006.10.08/MOD09A1.A2006281.h24v05.006.2015123101153.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2006.10.16/MOD09A1.A2006289.h24v05.006.2015123113851.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2006.10.24/MOD09A1.A2006297.h24v05.006.2015123111153.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2006.11.01/MOD09A1.A2006305.h24v05.006.2015124091906.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2006.11.09/MOD09A1.A2006313.h24v05.006.2015124075603.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2006.11.17/MOD09A1.A2006321.h24v05.006.2015124080255.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2006.11.25/MOD09A1.A2006329.h24v05.006.2015124085316.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2006.12.03/MOD09A1.A2006337.h24v05.006.2015125104906.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2006.12.11/MOD09A1.A2006345.h24v05.006.2015125105507.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2006.12.19/MOD09A1.A2006353.h24v05.006.2015125121947.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2006.12.27/MOD09A1.A2006361.h24v05.006.2015125111124.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2007.01.01/MOD09A1.A2007001.h24v05.006.2015126043024.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2007.01.09/MOD09A1.A2007009.h24v05.006.2015126114625.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2007.01.17/MOD09A1.A2007017.h24v05.006.2015126124332.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2007.01.25/MOD09A1.A2007025.h24v05.006.2015126114328.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2007.02.02/MOD09A1.A2007033.h24v05.006.2015126234450.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2007.02.10/MOD09A1.A2007041.h24v05.006.2015128063028.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2007.02.18/MOD09A1.A2007049.h24v05.006.2015128063228.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2007.02.26/MOD09A1.A2007057.h24v05.006.2015128222237.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2007.03.06/MOD09A1.A2007065.h24v05.006.2015128225843.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2007.03.14/MOD09A1.A2007073.h24v05.006.2015129090204.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2007.03.22/MOD09A1.A2007081.h24v05.006.2015129095827.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2007.03.30/MOD09A1.A2007089.h24v05.006.2015129094038.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2007.04.07/MOD09A1.A2007097.h24v05.006.2015129102427.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2007.04.15/MOD09A1.A2007105.h24v05.006.2015130095453.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2007.04.23/MOD09A1.A2007113.h24v05.006.2015130225508.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2007.05.01/MOD09A1.A2007121.h24v05.006.2015130193701.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2007.05.09/MOD09A1.A2007129.h24v05.006.2015130194044.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2007.05.17/MOD09A1.A2007137.h24v05.006.2015131082123.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2007.05.25/MOD09A1.A2007145.h24v05.006.2015131231954.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2007.06.02/MOD09A1.A2007153.h24v05.006.2015131231854.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2007.06.10/MOD09A1.A2007161.h24v05.006.2015132003053.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2007.06.18/MOD09A1.A2007169.h24v05.006.2015131223828.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2007.06.26/MOD09A1.A2007177.h24v05.006.2015132205153.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2007.07.04/MOD09A1.A2007185.h24v05.006.2015133100210.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2007.07.12/MOD09A1.A2007193.h24v05.006.2015133111353.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2007.07.20/MOD09A1.A2007201.h24v05.006.2015133105710.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2007.07.28/MOD09A1.A2007209.h24v05.006.2015133214442.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2007.08.05/MOD09A1.A2007217.h24v05.006.2015164051903.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2007.08.13/MOD09A1.A2007225.h24v05.006.2015164194559.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2007.08.21/MOD09A1.A2007233.h24v05.006.2015164193107.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2007.08.29/MOD09A1.A2007241.h24v05.006.2015164202657.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2007.09.06/MOD09A1.A2007249.h24v05.006.2015164222911.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2007.09.14/MOD09A1.A2007257.h24v05.006.2015166152735.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2007.09.22/MOD09A1.A2007265.h24v05.006.2015167072812.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2007.09.30/MOD09A1.A2007273.h24v05.006.2015167072650.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2007.10.08/MOD09A1.A2007281.h24v05.006.2015167084815.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2007.10.16/MOD09A1.A2007289.h24v05.006.2015167085218.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2007.10.24/MOD09A1.A2007297.h24v05.006.2015167223711.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2007.11.01/MOD09A1.A2007305.h24v05.006.2015168123307.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2007.11.09/MOD09A1.A2007313.h24v05.006.2015168102602.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2007.11.17/MOD09A1.A2007321.h24v05.006.2015168134040.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2007.11.25/MOD09A1.A2007329.h24v05.006.2015168121344.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2007.12.03/MOD09A1.A2007337.h24v05.006.2015168234343.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2007.12.11/MOD09A1.A2007345.h24v05.006.2015169064958.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2007.12.19/MOD09A1.A2007353.h24v05.006.2015169140302.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2007.12.27/MOD09A1.A2007361.h24v05.006.2015169064703.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2008.01.01/MOD09A1.A2008001.h24v05.006.2015169115116.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2008.01.09/MOD09A1.A2008009.h24v05.006.2015169114409.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2008.01.17/MOD09A1.A2008017.h24v05.006.2015169223435.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2008.01.25/MOD09A1.A2008025.h24v05.006.2015170074631.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2008.02.02/MOD09A1.A2008033.h24v05.006.2015170082631.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2008.02.10/MOD09A1.A2008041.h24v05.006.2015170084123.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2008.02.18/MOD09A1.A2008049.h24v05.006.2015171192239.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2008.02.26/MOD09A1.A2008057.h24v05.006.2015172170710.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2008.03.05/MOD09A1.A2008065.h24v05.006.2015172193105.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2008.03.13/MOD09A1.A2008073.h24v05.006.2015172213925.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2008.03.21/MOD09A1.A2008081.h24v05.006.2015172213510.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2008.03.29/MOD09A1.A2008089.h24v05.006.2015173033906.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2008.04.06/MOD09A1.A2008097.h24v05.006.2015173042307.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2008.04.14/MOD09A1.A2008105.h24v05.006.2015173093712.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2008.04.22/MOD09A1.A2008113.h24v05.006.2015173175007.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2008.04.30/MOD09A1.A2008121.h24v05.006.2015173201048.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2008.05.08/MOD09A1.A2008129.h24v05.006.2015174182217.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2008.05.16/MOD09A1.A2008137.h24v05.006.2015175130425.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2008.05.24/MOD09A1.A2008145.h24v05.006.2015175143632.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2008.06.01/MOD09A1.A2008153.h24v05.006.2015175144539.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2008.06.09/MOD09A1.A2008161.h24v05.006.2015175204539.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2008.06.17/MOD09A1.A2008169.h24v05.006.2015176111545.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2008.06.25/MOD09A1.A2008177.h24v05.006.2015176111300.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2008.07.03/MOD09A1.A2008185.h24v05.006.2015176105104.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2008.07.11/MOD09A1.A2008193.h24v05.006.2015176231143.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2008.07.19/MOD09A1.A2008201.h24v05.006.2015177070629.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2008.07.27/MOD09A1.A2008209.h24v05.006.2015177065439.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2008.08.04/MOD09A1.A2008217.h24v05.006.2015177083133.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2008.08.12/MOD09A1.A2008225.h24v05.006.2015178111334.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2008.08.20/MOD09A1.A2008233.h24v05.006.2015178201154.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2008.08.28/MOD09A1.A2008241.h24v05.006.2015178230339.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2008.09.05/MOD09A1.A2008249.h24v05.006.2015180004120.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2008.09.13/MOD09A1.A2008257.h24v05.006.2015180013132.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2008.09.21/MOD09A1.A2008265.h24v05.006.2015180010139.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2008.09.29/MOD09A1.A2008273.h24v05.006.2015180092141.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2008.10.07/MOD09A1.A2008281.h24v05.006.2015181012114.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2008.10.15/MOD09A1.A2008289.h24v05.006.2015181022747.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2008.10.23/MOD09A1.A2008297.h24v05.006.2015181021644.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2008.10.31/MOD09A1.A2008305.h24v05.006.2015181030631.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2008.11.08/MOD09A1.A2008313.h24v05.006.2015181180430.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2008.11.16/MOD09A1.A2008321.h24v05.006.2015181184731.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2008.11.24/MOD09A1.A2008329.h24v05.006.2015181184355.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2008.12.02/MOD09A1.A2008337.h24v05.006.2015184003738.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2008.12.10/MOD09A1.A2008345.h24v05.006.2015184044816.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2008.12.18/MOD09A1.A2008353.h24v05.006.2015184033436.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2008.12.26/MOD09A1.A2008361.h24v05.006.2015186050349.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2009.01.01/MOD09A1.A2009001.h24v05.006.2015186091708.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2009.01.09/MOD09A1.A2009009.h24v05.006.2015186100115.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2009.01.17/MOD09A1.A2009017.h24v05.006.2015186101248.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2009.01.25/MOD09A1.A2009025.h24v05.006.2015186100926.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2009.02.02/MOD09A1.A2009033.h24v05.006.2015186095646.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2009.02.10/MOD09A1.A2009041.h24v05.006.2015187103906.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2009.02.18/MOD09A1.A2009049.h24v05.006.2015187095034.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2009.02.26/MOD09A1.A2009057.h24v05.006.2015187101312.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2009.03.06/MOD09A1.A2009065.h24v05.006.2015187190727.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2009.03.14/MOD09A1.A2009073.h24v05.006.2015188121956.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2009.03.22/MOD09A1.A2009081.h24v05.006.2015188125358.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2009.03.30/MOD09A1.A2009089.h24v05.006.2015188123126.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2009.04.07/MOD09A1.A2009097.h24v05.006.2015188214043.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2009.04.15/MOD09A1.A2009105.h24v05.006.2015189190558.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2009.04.23/MOD09A1.A2009113.h24v05.006.2015189181822.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2009.05.01/MOD09A1.A2009121.h24v05.006.2015189173256.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2009.05.09/MOD09A1.A2009129.h24v05.006.2015189172819.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2009.05.17/MOD09A1.A2009137.h24v05.006.2015190004542.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2009.05.25/MOD09A1.A2009145.h24v05.006.2015190154021.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2009.06.02/MOD09A1.A2009153.h24v05.006.2015190154601.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2009.06.10/MOD09A1.A2009161.h24v05.006.2015190170900.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2009.06.18/MOD09A1.A2009169.h24v05.006.2015191020457.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2009.06.26/MOD09A1.A2009177.h24v05.006.2015191131324.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2009.07.04/MOD09A1.A2009185.h24v05.006.2015191132341.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2009.07.12/MOD09A1.A2009193.h24v05.006.2015191131219.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2009.07.20/MOD09A1.A2009201.h24v05.006.2015192175101.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2009.07.28/MOD09A1.A2009209.h24v05.006.2015192181127.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2009.08.05/MOD09A1.A2009217.h24v05.006.2015192174718.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2009.08.13/MOD09A1.A2009225.h24v05.006.2015192175038.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2009.08.21/MOD09A1.A2009233.h24v05.006.2015194181822.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2009.08.29/MOD09A1.A2009241.h24v05.006.2015194171247.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2009.09.06/MOD09A1.A2009249.h24v05.006.2015194184624.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2009.09.14/MOD09A1.A2009257.h24v05.006.2015195022158.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2009.09.22/MOD09A1.A2009265.h24v05.006.2015195024515.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2009.09.30/MOD09A1.A2009273.h24v05.006.2015195162018.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2009.10.08/MOD09A1.A2009281.h24v05.006.2015195161113.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2009.10.16/MOD09A1.A2009289.h24v05.006.2015195172009.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2009.10.24/MOD09A1.A2009297.h24v05.006.2015196022109.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2009.11.01/MOD09A1.A2009305.h24v05.006.2015196163308.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2009.11.09/MOD09A1.A2009313.h24v05.006.2015196172315.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2009.11.17/MOD09A1.A2009321.h24v05.006.2015196174507.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2009.11.25/MOD09A1.A2009329.h24v05.006.2015197001920.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2009.12.03/MOD09A1.A2009337.h24v05.006.2015197103007.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2009.12.11/MOD09A1.A2009345.h24v05.006.2015197110817.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2009.12.19/MOD09A1.A2009353.h24v05.006.2015197105910.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2009.12.27/MOD09A1.A2009361.h24v05.006.2015198070845.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2010.01.01/MOD09A1.A2010001.h24v05.006.2015198080916.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2010.01.09/MOD09A1.A2010009.h24v05.006.2015198141419.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2010.01.17/MOD09A1.A2010017.h24v05.006.2015198134717.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2010.01.25/MOD09A1.A2010025.h24v05.006.2015198233907.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2010.02.02/MOD09A1.A2010033.h24v05.006.2015199154410.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2010.02.10/MOD09A1.A2010041.h24v05.006.2015199153842.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2010.02.18/MOD09A1.A2010049.h24v05.006.2015199154652.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2010.02.26/MOD09A1.A2010057.h24v05.006.2015200062016.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2010.03.06/MOD09A1.A2010065.h24v05.006.2015205185644.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2010.03.14/MOD09A1.A2010073.h24v05.006.2015206113629.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2010.03.22/MOD09A1.A2010081.h24v05.006.2015206100408.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2010.03.30/MOD09A1.A2010089.h24v05.006.2015206114611.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2010.04.07/MOD09A1.A2010097.h24v05.006.2015206122132.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2010.04.15/MOD09A1.A2010105.h24v05.006.2015207022606.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2010.04.23/MOD09A1.A2010113.h24v05.006.2015207111320.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2010.05.01/MOD09A1.A2010121.h24v05.006.2015207183946.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2010.05.09/MOD09A1.A2010129.h24v05.006.2015207194057.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2010.05.17/MOD09A1.A2010137.h24v05.006.2015207215131.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2010.05.25/MOD09A1.A2010145.h24v05.006.2015208041441.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2010.06.02/MOD09A1.A2010153.h24v05.006.2015208175511.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2010.06.10/MOD09A1.A2010161.h24v05.006.2015208214554.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2010.06.18/MOD09A1.A2010169.h24v05.006.2015208214628.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2010.06.26/MOD09A1.A2010177.h24v05.006.2015208232810.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2010.07.04/MOD09A1.A2010185.h24v05.006.2015208231310.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2010.07.12/MOD09A1.A2010193.h24v05.006.2015209053901.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2010.07.20/MOD09A1.A2010201.h24v05.006.2015209134828.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2010.07.28/MOD09A1.A2010209.h24v05.006.2015209184629.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2010.08.05/MOD09A1.A2010217.h24v05.006.2015209185230.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2010.08.13/MOD09A1.A2010225.h24v05.006.2015210010803.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2010.08.21/MOD09A1.A2010233.h24v05.006.2015210124533.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2010.08.29/MOD09A1.A2010241.h24v05.006.2015210140550.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2010.09.06/MOD09A1.A2010249.h24v05.006.2015210133848.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2010.09.14/MOD09A1.A2010257.h24v05.006.2015210233030.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2010.09.22/MOD09A1.A2010265.h24v05.006.2015211062057.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2010.09.30/MOD09A1.A2010273.h24v05.006.2015211083458.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2010.10.08/MOD09A1.A2010281.h24v05.006.2015211180617.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2010.10.16/MOD09A1.A2010289.h24v05.006.2015212015023.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2010.10.24/MOD09A1.A2010297.h24v05.006.2015212025331.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2010.11.01/MOD09A1.A2010305.h24v05.006.2015212190540.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2010.11.09/MOD09A1.A2010313.h24v05.006.2015213102913.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2010.11.17/MOD09A1.A2010321.h24v05.006.2015213113836.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2010.11.25/MOD09A1.A2010329.h24v05.006.2015213123606.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2010.12.03/MOD09A1.A2010337.h24v05.006.2015213124430.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2010.12.11/MOD09A1.A2010345.h24v05.006.2015213202743.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2010.12.19/MOD09A1.A2010353.h24v05.006.2015214054618.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2010.12.27/MOD09A1.A2010361.h24v05.006.2015216044431.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2011.01.01/MOD09A1.A2011001.h24v05.006.2015216140420.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2011.01.09/MOD09A1.A2011009.h24v05.006.2015216112303.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2011.01.17/MOD09A1.A2011017.h24v05.006.2015216173948.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2011.01.25/MOD09A1.A2011025.h24v05.006.2015216185747.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2011.02.02/MOD09A1.A2011033.h24v05.006.2015216170201.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2011.02.10/MOD09A1.A2011041.h24v05.006.2015216135411.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2011.02.18/MOD09A1.A2011049.h24v05.006.2015216172206.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2011.02.26/MOD09A1.A2011057.h24v05.006.2015216165922.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2011.03.06/MOD09A1.A2011065.h24v05.006.2015217013710.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2011.03.14/MOD09A1.A2011073.h24v05.006.2015217101057.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2011.03.22/MOD09A1.A2011081.h24v05.006.2015217122917.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2011.03.30/MOD09A1.A2011089.h24v05.006.2015217193534.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2011.04.07/MOD09A1.A2011097.h24v05.006.2015218115240.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2011.04.15/MOD09A1.A2011105.h24v05.006.2015218103118.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2011.04.23/MOD09A1.A2011113.h24v05.006.2015218124801.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2011.05.01/MOD09A1.A2011121.h24v05.006.2015218141452.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2011.05.09/MOD09A1.A2011129.h24v05.006.2015218222703.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2011.05.17/MOD09A1.A2011137.h24v05.006.2015219134612.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2011.05.25/MOD09A1.A2011145.h24v05.006.2015219145433.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2011.06.02/MOD09A1.A2011153.h24v05.006.2015219174830.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2011.06.10/MOD09A1.A2011161.h24v05.006.2015219151205.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2011.06.18/MOD09A1.A2011169.h24v05.006.2015219230313.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2011.06.26/MOD09A1.A2011177.h24v05.006.2015220112237.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2011.07.04/MOD09A1.A2011185.h24v05.006.2015220091650.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2011.07.12/MOD09A1.A2011193.h24v05.006.2015220134621.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2011.07.20/MOD09A1.A2011201.h24v05.006.2015220142057.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2011.07.28/MOD09A1.A2011209.h24v05.006.2015221003729.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2011.08.05/MOD09A1.A2011217.h24v05.006.2015221113605.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2011.08.13/MOD09A1.A2011225.h24v05.006.2015221142221.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2011.08.21/MOD09A1.A2011233.h24v05.006.2015221153854.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2011.08.29/MOD09A1.A2011241.h24v05.006.2015221160436.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2011.09.06/MOD09A1.A2011249.h24v05.006.2015222010441.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2011.09.14/MOD09A1.A2011257.h24v05.006.2015222104524.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2011.09.22/MOD09A1.A2011265.h24v05.006.2015222133449.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2011.09.30/MOD09A1.A2011273.h24v05.006.2015222135956.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2011.10.08/MOD09A1.A2011281.h24v05.006.2015222160825.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2011.10.16/MOD09A1.A2011289.h24v05.006.2015223011256.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2011.10.24/MOD09A1.A2011297.h24v05.006.2015223162215.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2011.11.01/MOD09A1.A2011305.h24v05.006.2015229130130.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2011.11.09/MOD09A1.A2011313.h24v05.006.2015229132706.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2011.11.17/MOD09A1.A2011321.h24v05.006.2015229133616.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2011.11.25/MOD09A1.A2011329.h24v05.006.2015229150848.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2011.12.03/MOD09A1.A2011337.h24v05.006.2015230013119.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2011.12.11/MOD09A1.A2011345.h24v05.006.2015230033811.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2011.12.19/MOD09A1.A2011353.h24v05.006.2015230051313.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2011.12.27/MOD09A1.A2011361.h24v05.006.2015230020707.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2012.01.01/MOD09A1.A2012001.h24v05.006.2015230180936.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2012.01.09/MOD09A1.A2012009.h24v05.006.2015236194156.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2012.01.17/MOD09A1.A2012017.h24v05.006.2015237120225.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2012.01.25/MOD09A1.A2012025.h24v05.006.2015237124622.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2012.02.02/MOD09A1.A2012033.h24v05.006.2015237130754.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2012.02.10/MOD09A1.A2012041.h24v05.006.2015237232714.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2012.02.18/MOD09A1.A2012049.h24v05.006.2015238141259.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2012.02.26/MOD09A1.A2012057.h24v05.006.2015238171502.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2012.03.05/MOD09A1.A2012065.h24v05.006.2015238191613.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2012.03.13/MOD09A1.A2012073.h24v05.006.2015238175510.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2012.03.21/MOD09A1.A2012081.h24v05.006.2015239173931.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2012.03.29/MOD09A1.A2012089.h24v05.006.2015240025126.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2012.04.06/MOD09A1.A2012097.h24v05.006.2015240045740.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2012.04.14/MOD09A1.A2012105.h24v05.006.2015240142120.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2012.04.22/MOD09A1.A2012113.h24v05.006.2015240162516.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2012.04.30/MOD09A1.A2012121.h24v05.006.2015240224328.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2012.05.08/MOD09A1.A2012129.h24v05.006.2015246113705.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2012.05.16/MOD09A1.A2012137.h24v05.006.2015246072256.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2012.05.24/MOD09A1.A2012145.h24v05.006.2015246135305.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2012.06.01/MOD09A1.A2012153.h24v05.006.2015246133756.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2012.06.09/MOD09A1.A2012161.h24v05.006.2015246141710.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2012.06.17/MOD09A1.A2012169.h24v05.006.2015247091249.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2012.06.25/MOD09A1.A2012177.h24v05.006.2015247052947.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2012.07.03/MOD09A1.A2012185.h24v05.006.2015247121613.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2012.07.11/MOD09A1.A2012193.h24v05.006.2015247115218.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2012.07.19/MOD09A1.A2012201.h24v05.006.2015247202845.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2012.07.27/MOD09A1.A2012209.h24v05.006.2015248132522.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2012.08.04/MOD09A1.A2012217.h24v05.006.2015248134055.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2012.08.12/MOD09A1.A2012225.h24v05.006.2015248130638.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2012.08.20/MOD09A1.A2012233.h24v05.006.2015248162437.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2012.08.28/MOD09A1.A2012241.h24v05.006.2015249021749.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2012.09.05/MOD09A1.A2012249.h24v05.006.2015249155018.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2012.09.13/MOD09A1.A2012257.h24v05.006.2015249162949.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2012.09.21/MOD09A1.A2012265.h24v05.006.2015249172431.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2012.09.29/MOD09A1.A2012273.h24v05.006.2015249185434.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2012.10.07/MOD09A1.A2012281.h24v05.006.2015250012914.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2012.10.15/MOD09A1.A2012289.h24v05.006.2015251061159.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2012.10.23/MOD09A1.A2012297.h24v05.006.2015251170419.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2012.10.31/MOD09A1.A2012305.h24v05.006.2015251175036.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2012.11.08/MOD09A1.A2012313.h24v05.006.2015251165253.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2012.11.16/MOD09A1.A2012321.h24v05.006.2015252134249.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2012.11.24/MOD09A1.A2012329.h24v05.006.2015252222658.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2012.12.02/MOD09A1.A2012337.h24v05.006.2015253025037.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2012.12.10/MOD09A1.A2012345.h24v05.006.2015253032547.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2012.12.18/MOD09A1.A2012353.h24v05.006.2015253042531.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2012.12.26/MOD09A1.A2012361.h24v05.006.2015254022203.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2013.01.01/MOD09A1.A2013001.h24v05.006.2015254190326.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2013.01.09/MOD09A1.A2013009.h24v05.006.2015254165926.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2013.01.17/MOD09A1.A2013017.h24v05.006.2015254165541.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2013.01.25/MOD09A1.A2013025.h24v05.006.2015254182609.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2013.02.02/MOD09A1.A2013033.h24v05.006.2015255201003.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2013.02.10/MOD09A1.A2013041.h24v05.006.2015256103416.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2013.02.18/MOD09A1.A2013049.h24v05.006.2015256202322.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2013.02.26/MOD09A1.A2013057.h24v05.006.2015256201322.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2013.03.06/MOD09A1.A2013065.h24v05.006.2015256210859.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2013.03.14/MOD09A1.A2013073.h24v05.006.2015256194515.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2013.03.22/MOD09A1.A2013081.h24v05.006.2015256233111.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2013.03.30/MOD09A1.A2013089.h24v05.006.2015257112127.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2013.04.07/MOD09A1.A2013097.h24v05.006.2015257141408.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2013.04.15/MOD09A1.A2013105.h24v05.006.2015257133448.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2013.04.23/MOD09A1.A2013113.h24v05.006.2015258193158.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2013.05.01/MOD09A1.A2013121.h24v05.006.2015260134644.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2013.05.09/MOD09A1.A2013129.h24v05.006.2015260165024.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2013.05.17/MOD09A1.A2013137.h24v05.006.2015260202334.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2013.05.25/MOD09A1.A2013145.h24v05.006.2015260204917.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2013.06.02/MOD09A1.A2013153.h24v05.006.2015261222222.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2013.06.10/MOD09A1.A2013161.h24v05.006.2015262182349.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2013.06.18/MOD09A1.A2013169.h24v05.006.2015262201551.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2013.06.26/MOD09A1.A2013177.h24v05.006.2015262192001.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2013.07.04/MOD09A1.A2013185.h24v05.006.2015262203138.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2013.07.12/MOD09A1.A2013193.h24v05.006.2015263212041.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2013.07.20/MOD09A1.A2013201.h24v05.006.2015264100002.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2013.07.28/MOD09A1.A2013209.h24v05.006.2015267192537.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2013.08.05/MOD09A1.A2013217.h24v05.006.2015267180540.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2013.08.13/MOD09A1.A2013225.h24v05.006.2015267193415.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2013.08.21/MOD09A1.A2013233.h24v05.006.2015268013136.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2013.08.29/MOD09A1.A2013241.h24v05.006.2015268094253.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2013.09.06/MOD09A1.A2013249.h24v05.006.2015268113130.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2013.09.14/MOD09A1.A2013257.h24v05.006.2015268115100.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2013.09.22/MOD09A1.A2013265.h24v05.006.2015268102115.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2013.09.30/MOD09A1.A2013273.h24v05.006.2015269081826.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2013.10.08/MOD09A1.A2013281.h24v05.006.2015269223025.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2013.10.16/MOD09A1.A2013289.h24v05.006.2015270011830.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2013.10.24/MOD09A1.A2013297.h24v05.006.2015270021802.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2013.11.01/MOD09A1.A2013305.h24v05.006.2015270025021.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2013.11.09/MOD09A1.A2013313.h24v05.006.2015270224908.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2013.11.17/MOD09A1.A2013321.h24v05.006.2015271123632.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2013.11.25/MOD09A1.A2013329.h24v05.006.2015271135155.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2013.12.03/MOD09A1.A2013337.h24v05.006.2015271151006.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2013.12.11/MOD09A1.A2013345.h24v05.006.2015271171217.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2013.12.19/MOD09A1.A2013353.h24v05.006.2015271231214.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2013.12.27/MOD09A1.A2013361.h24v05.006.2015272063948.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2014.01.01/MOD09A1.A2014001.h24v05.006.2015272083934.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2014.01.09/MOD09A1.A2014009.h24v05.006.2015272101403.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2014.01.17/MOD09A1.A2014017.h24v05.006.2015272090311.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2014.01.25/MOD09A1.A2014025.h24v05.006.2015274025005.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2014.02.02/MOD09A1.A2014033.h24v05.006.2015275013436.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2014.02.10/MOD09A1.A2014041.h24v05.006.2015275024946.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2014.02.18/MOD09A1.A2014049.h24v05.006.2015275022839.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2014.02.26/MOD09A1.A2014057.h24v05.006.2015275023937.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2014.03.06/MOD09A1.A2014065.h24v05.006.2015276025429.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2014.03.14/MOD09A1.A2014073.h24v05.006.2015276213948.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2014.03.22/MOD09A1.A2014081.h24v05.006.2015277015809.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2014.03.30/MOD09A1.A2014089.h24v05.006.2015277012116.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2014.04.07/MOD09A1.A2014097.h24v05.006.2015277000437.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2014.04.15/MOD09A1.A2014105.h24v05.006.2015279032601.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2014.04.23/MOD09A1.A2014113.h24v05.006.2015279165713.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2014.05.01/MOD09A1.A2014121.h24v05.006.2015281135848.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2014.05.09/MOD09A1.A2014129.h24v05.006.2015281142855.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2014.05.17/MOD09A1.A2014137.h24v05.006.2015281144519.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2014.05.25/MOD09A1.A2014145.h24v05.006.2015287011927.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2014.06.02/MOD09A1.A2014153.h24v05.006.2015288010059.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2014.06.10/MOD09A1.A2014161.h24v05.006.2015288112820.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2014.06.18/MOD09A1.A2014169.h24v05.006.2015288085120.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2014.06.26/MOD09A1.A2014177.h24v05.006.2015288091510.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2014.07.04/MOD09A1.A2014185.h24v05.006.2015288103741.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2014.07.12/MOD09A1.A2014193.h24v05.006.2015289021756.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2014.07.20/MOD09A1.A2014201.h24v05.006.2015289034924.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2014.07.28/MOD09A1.A2014209.h24v05.006.2015289020359.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2014.08.05/MOD09A1.A2014217.h24v05.006.2015289071156.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2014.08.13/MOD09A1.A2014225.h24v05.006.2015289103007.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2014.08.21/MOD09A1.A2014233.h24v05.006.2015289191943.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2014.08.29/MOD09A1.A2014241.h24v05.006.2015289223154.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2014.09.06/MOD09A1.A2014249.h24v05.006.2015289223145.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2014.09.14/MOD09A1.A2014257.h24v05.006.2015290233059.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2014.09.22/MOD09A1.A2014265.h24v05.006.2015291101348.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2014.09.30/MOD09A1.A2014273.h24v05.006.2015291114701.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2014.10.08/MOD09A1.A2014281.h24v05.006.2015291144840.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2014.10.16/MOD09A1.A2014289.h24v05.006.2015291145227.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2014.10.24/MOD09A1.A2014297.h24v05.006.2015291222016.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2014.11.01/MOD09A1.A2014305.h24v05.006.2015292100200.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2014.11.09/MOD09A1.A2014313.h24v05.006.2015292094939.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2014.11.17/MOD09A1.A2014321.h24v05.006.2015292101752.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2014.11.25/MOD09A1.A2014329.h24v05.006.2015292110822.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2014.12.03/MOD09A1.A2014337.h24v05.006.2015293182144.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2014.12.11/MOD09A1.A2014345.h24v05.006.2015294130639.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2014.12.19/MOD09A1.A2014353.h24v05.006.2015294021220.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2014.12.27/MOD09A1.A2014361.h24v05.006.2015295041450.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2015.01.01/MOD09A1.A2015001.h24v05.006.2015295091918.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2015.01.09/MOD09A1.A2015009.h24v05.006.2015295140743.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2015.01.17/MOD09A1.A2015017.h24v05.006.2015295135707.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2015.01.25/MOD09A1.A2015025.h24v05.006.2015295155524.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2015.02.02/MOD09A1.A2015033.h24v05.006.2015296002858.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2015.02.10/MOD09A1.A2015041.h24v05.006.2015296121502.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2015.02.18/MOD09A1.A2015049.h24v05.006.2015296182617.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2015.02.26/MOD09A1.A2015057.h24v05.006.2015296191851.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2015.03.06/MOD09A1.A2015065.h24v05.006.2015297052034.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2015.03.14/MOD09A1.A2015073.h24v05.006.2015298013500.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2015.03.22/MOD09A1.A2015081.h24v05.006.2015298194750.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2015.03.30/MOD09A1.A2015089.h24v05.006.2015298195513.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2015.04.07/MOD09A1.A2015097.h24v05.006.2015298230609.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2015.04.15/MOD09A1.A2015105.h24v05.006.2015299024032.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2015.04.23/MOD09A1.A2015113.h24v05.006.2015299013204.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2015.05.01/MOD09A1.A2015121.h24v05.006.2015299045509.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2015.05.09/MOD09A1.A2015129.h24v05.006.2015299173224.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2015.05.17/MOD09A1.A2015137.h24v05.006.2015299190757.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2015.05.25/MOD09A1.A2015145.h24v05.006.2015299184753.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2015.06.02/MOD09A1.A2015153.h24v05.006.2015301032116.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2015.06.10/MOD09A1.A2015161.h24v05.006.2015301101724.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2015.06.18/MOD09A1.A2015169.h24v05.006.2015301110516.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2015.06.26/MOD09A1.A2015177.h24v05.006.2015301223602.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2015.07.04/MOD09A1.A2015185.h24v05.006.2015302014650.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2015.07.12/MOD09A1.A2015193.h24v05.006.2015303065726.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2015.07.20/MOD09A1.A2015201.h24v05.006.2015304035321.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2015.07.28/MOD09A1.A2015209.h24v05.006.2015304065250.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2015.08.05/MOD09A1.A2015217.h24v05.006.2015304063404.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2015.08.13/MOD09A1.A2015225.h24v05.006.2015305172004.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2015.08.21/MOD09A1.A2015233.h24v05.006.2015306012554.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2015.08.29/MOD09A1.A2015241.h24v05.006.2015306005921.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2015.09.06/MOD09A1.A2015249.h24v05.006.2015306021743.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2015.09.14/MOD09A1.A2015257.h24v05.006.2015306021919.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2015.09.22/MOD09A1.A2015265.h24v05.006.2015306194009.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2015.09.30/MOD09A1.A2015273.h24v05.006.2015307100731.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2015.10.08/MOD09A1.A2015281.h24v05.006.2015307102947.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2015.10.16/MOD09A1.A2015289.h24v05.006.2015317184207.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2015.10.24/MOD09A1.A2015297.h24v05.006.2015318022836.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2015.11.01/MOD09A1.A2015305.h24v05.006.2015334220745.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2015.11.09/MOD09A1.A2015313.h24v05.006.2015335181045.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2015.11.17/MOD09A1.A2015321.h24v05.006.2015341234828.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2015.11.25/MOD09A1.A2015329.h24v05.006.2015343185520.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2015.12.03/MOD09A1.A2015337.h24v05.006.2016004203933.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2015.12.11/MOD09A1.A2015345.h24v05.006.2016004212518.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2015.12.19/MOD09A1.A2015353.h24v05.006.2016004215927.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2015.12.27/MOD09A1.A2015361.h24v05.006.2016007230754.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2016.01.01/MOD09A1.A2016001.h24v05.006.2016011194220.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2016.01.09/MOD09A1.A2016009.h24v05.006.2016029115000.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2016.01.17/MOD09A1.A2016017.h24v05.006.2016026072117.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2016.01.25/MOD09A1.A2016025.h24v05.006.2016035161726.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2016.02.02/MOD09A1.A2016033.h24v05.006.2016043141239.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2016.02.10/MOD09A1.A2016041.h24v05.006.2016050072442.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2016.02.18/MOD09A1.A2016049.h24v05.006.2016109160626.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2016.02.26/MOD09A1.A2016057.h24v05.006.2016109162557.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2016.03.05/MOD09A1.A2016065.h24v05.006.2016110191425.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2016.03.13/MOD09A1.A2016073.h24v05.006.2016110193931.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2016.03.21/MOD09A1.A2016081.h24v05.006.2016110195829.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2016.03.29/MOD09A1.A2016089.h24v05.006.2016110201817.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2016.04.06/MOD09A1.A2016097.h24v05.006.2016106221332.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2016.04.14/MOD09A1.A2016105.h24v05.006.2016114075711.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2016.04.22/MOD09A1.A2016113.h24v05.006.2016123132740.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2016.04.30/MOD09A1.A2016121.h24v05.006.2016130150348.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2016.05.08/MOD09A1.A2016129.h24v05.006.2016140152327.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2016.05.16/MOD09A1.A2016137.h24v05.006.2016147144237.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2016.05.24/MOD09A1.A2016145.h24v05.006.2016155181202.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2016.06.01/MOD09A1.A2016153.h24v05.006.2016166184019.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2016.06.09/MOD09A1.A2016161.h24v05.006.2016170073212.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2016.06.17/MOD09A1.A2016169.h24v05.006.2016184050728.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2016.06.25/MOD09A1.A2016177.h24v05.006.2016188012254.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2016.07.03/MOD09A1.A2016185.h24v05.006.2016194193803.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2016.07.11/MOD09A1.A2016193.h24v05.006.2016202083654.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2016.07.19/MOD09A1.A2016201.h24v05.006.2016215130400.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2016.07.27/MOD09A1.A2016209.h24v05.006.2016222122630.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2016.08.04/MOD09A1.A2016217.h24v05.006.2016226075021.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2016.08.12/MOD09A1.A2016225.h24v05.006.2016235055801.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2016.08.20/MOD09A1.A2016233.h24v05.006.2016243123055.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2016.08.28/MOD09A1.A2016241.h24v05.006.2016250085155.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2016.09.05/MOD09A1.A2016249.h24v05.006.2016258072201.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2016.09.13/MOD09A1.A2016257.h24v05.006.2016267015709.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2016.09.21/MOD09A1.A2016265.h24v05.006.2016274184325.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2016.09.29/MOD09A1.A2016273.h24v05.006.2016282072356.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2016.10.07/MOD09A1.A2016281.h24v05.006.2016292110136.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2016.10.15/MOD09A1.A2016289.h24v05.006.2016302012152.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2016.10.23/MOD09A1.A2016297.h24v05.006.2016306082517.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2016.10.31/MOD09A1.A2016305.h24v05.006.2016314202048.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2016.11.08/MOD09A1.A2016313.h24v05.006.2016322094344.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2016.11.16/MOD09A1.A2016321.h24v05.006.2016340134207.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2016.11.24/MOD09A1.A2016329.h24v05.006.2016340143020.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2016.12.02/MOD09A1.A2016337.h24v05.006.2016346105258.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2016.12.10/MOD09A1.A2016345.h24v05.006.2016357094034.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2016.12.18/MOD09A1.A2016353.h24v05.006.2016362135707.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2016.12.26/MOD09A1.A2016361.h24v05.006.2017010135441.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2017.01.01/MOD09A1.A2017001.h24v05.006.2017017151426.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2017.01.09/MOD09A1.A2017009.h24v05.006.2017021025010.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2017.01.17/MOD09A1.A2017017.h24v05.006.2017026122619.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2017.01.25/MOD09A1.A2017025.h24v05.006.2017034123249.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2017.02.02/MOD09A1.A2017033.h24v05.006.2017044220233.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2017.02.10/MOD09A1.A2017041.h24v05.006.2017053110307.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2017.02.18/MOD09A1.A2017049.h24v05.006.2017059075916.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2017.02.26/MOD09A1.A2017057.h24v05.006.2017066081655.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2017.03.06/MOD09A1.A2017065.h24v05.006.2017074144252.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2017.03.14/MOD09A1.A2017073.h24v05.006.2017082160841.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2017.03.22/MOD09A1.A2017081.h24v05.006.2017090162130.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2017.03.30/MOD09A1.A2017089.h24v05.006.2017098031025.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2017.04.07/MOD09A1.A2017097.h24v05.006.2017116180114.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2017.04.15/MOD09A1.A2017105.h24v05.006.2017114193720.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2017.04.23/MOD09A1.A2017113.h24v05.006.2017128142859.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2017.05.01/MOD09A1.A2017121.h24v05.006.2017131132219.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2017.05.09/MOD09A1.A2017129.h24v05.006.2017139025809.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2017.05.17/MOD09A1.A2017137.h24v05.006.2017146031126.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2017.05.25/MOD09A1.A2017145.h24v05.006.2017154030831.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2017.06.02/MOD09A1.A2017153.h24v05.006.2017163190131.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2017.06.10/MOD09A1.A2017161.h24v05.006.2017172161439.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2017.06.18/MOD09A1.A2017169.h24v05.006.2017178120526.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2017.06.26/MOD09A1.A2017177.h24v05.006.2017187225005.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2017.07.04/MOD09A1.A2017185.h24v05.006.2017194105644.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2017.07.12/MOD09A1.A2017193.h24v05.006.2017202031519.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2017.07.20/MOD09A1.A2017201.h24v05.006.2017210034027.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2017.07.28/MOD09A1.A2017209.h24v05.006.2017218045551.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2017.08.05/MOD09A1.A2017217.h24v05.006.2017234151116.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2017.08.13/MOD09A1.A2017225.h24v05.006.2017234032341.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2017.08.21/MOD09A1.A2017233.h24v05.006.2017249210014.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2017.08.29/MOD09A1.A2017241.h24v05.006.2017250140504.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2017.09.06/MOD09A1.A2017249.h24v05.006.2017262125318.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2017.09.14/MOD09A1.A2017257.h24v05.006.2017266040447.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2017.09.22/MOD09A1.A2017265.h24v05.006.2017276172219.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2017.09.30/MOD09A1.A2017273.h24v05.006.2017282033118.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2017.10.08/MOD09A1.A2017281.h24v05.006.2017290130147.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2017.10.16/MOD09A1.A2017289.h24v05.006.2017298033057.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2017.10.24/MOD09A1.A2017297.h24v05.006.2017310191110.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2017.11.01/MOD09A1.A2017305.h24v05.006.2017314041945.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2017.11.09/MOD09A1.A2017313.h24v05.006.2017325163006.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2017.11.17/MOD09A1.A2017321.h24v05.006.2017331213035.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2017.11.25/MOD09A1.A2017329.h24v05.006.2017338031715.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2017.12.03/MOD09A1.A2017337.h24v05.006.2017346034102.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2017.12.11/MOD09A1.A2017345.h24v05.006.2017354033807.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2017.12.19/MOD09A1.A2017353.h24v05.006.2017362034654.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2017.12.27/MOD09A1.A2017361.h24v05.006.2018005034818.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2018.01.01/MOD09A1.A2018001.h24v05.006.2018011144850.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2018.01.09/MOD09A1.A2018009.h24v05.006.2018018033741.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2018.01.17/MOD09A1.A2018017.h24v05.006.2018026031951.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2018.01.25/MOD09A1.A2018025.h24v05.006.2018034033326.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2018.02.02/MOD09A1.A2018033.h24v05.006.2018042035233.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2018.02.10/MOD09A1.A2018041.h24v05.006.2018050032731.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2018.02.18/MOD09A1.A2018049.h24v05.006.2018060011156.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2018.02.26/MOD09A1.A2018057.h24v05.006.2018066215931.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2018.03.06/MOD09A1.A2018065.h24v05.006.2018074035624.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2018.03.14/MOD09A1.A2018073.h24v05.006.2018082192046.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2018.03.22/MOD09A1.A2018081.h24v05.006.2018090030629.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2018.03.30/MOD09A1.A2018089.h24v05.006.2018098033318.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2018.04.07/MOD09A1.A2018097.h24v05.006.2018106033935.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2018.04.15/MOD09A1.A2018105.h24v05.006.2018114035847.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2018.04.23/MOD09A1.A2018113.h24v05.006.2018122035849.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2018.05.01/MOD09A1.A2018121.h24v05.006.2018130035350.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2018.05.09/MOD09A1.A2018129.h24v05.006.2018151145133.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2018.05.17/MOD09A1.A2018137.h24v05.006.2018151150539.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2018.05.25/MOD09A1.A2018145.h24v05.006.2018154035518.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2018.06.02/MOD09A1.A2018153.h24v05.006.2018162042550.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2018.06.10/MOD09A1.A2018161.h24v05.006.2018169142553.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2018.06.18/MOD09A1.A2018169.h24v05.006.2018178035428.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2018.06.26/MOD09A1.A2018177.h24v05.006.2018186223607.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2018.07.04/MOD09A1.A2018185.h24v05.006.2018197142748.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2018.07.12/MOD09A1.A2018193.h24v05.006.2018202040838.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2018.07.20/MOD09A1.A2018201.h24v05.006.2018210041206.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2018.07.28/MOD09A1.A2018209.h24v05.006.2018218215451.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2018.08.05/MOD09A1.A2018217.h24v05.006.2018227165322.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2018.08.13/MOD09A1.A2018225.h24v05.006.2018234040633.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2018.08.21/MOD09A1.A2018233.h24v05.006.2018242035420.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2018.08.29/MOD09A1.A2018241.h24v05.006.2018250042521.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2018.09.06/MOD09A1.A2018249.h24v05.006.2018258035256.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2018.09.14/MOD09A1.A2018257.h24v05.006.2018266035927.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2018.09.22/MOD09A1.A2018265.h24v05.006.2018282171825.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2018.09.30/MOD09A1.A2018273.h24v05.006.2018290185841.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2018.10.08/MOD09A1.A2018281.h24v05.006.2018295151745.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2018.10.16/MOD09A1.A2018289.h24v05.006.2018302173246.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2018.10.24/MOD09A1.A2018297.h24v05.006.2018312183145.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2018.11.01/MOD09A1.A2018305.h24v05.006.2018316152813.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2018.11.09/MOD09A1.A2018313.h24v05.006.2018322034142.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2018.11.17/MOD09A1.A2018321.h24v05.006.2018330171525.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2018.11.25/MOD09A1.A2018329.h24v05.006.2018338032755.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2018.12.03/MOD09A1.A2018337.h24v05.006.2018348144253.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2018.12.11/MOD09A1.A2018345.h24v05.006.2018355162218.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2018.12.19/MOD09A1.A2018353.h24v05.006.2019032175858.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2018.12.27/MOD09A1.A2018361.h24v05.006.2019009093804.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2019.01.01/MOD09A1.A2019001.h24v05.006.2019010204551.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2019.01.09/MOD09A1.A2019009.h24v05.006.2019022164805.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2019.01.17/MOD09A1.A2019017.h24v05.006.2019030120257.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2019.01.25/MOD09A1.A2019025.h24v05.006.2019037210545.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2019.02.02/MOD09A1.A2019033.h24v05.006.2019042144702.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2019.02.10/MOD09A1.A2019041.h24v05.006.2019051023757.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2019.02.18/MOD09A1.A2019049.h24v05.006.2019059192706.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2019.02.26/MOD09A1.A2019057.h24v05.006.2019084162853.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2019.03.06/MOD09A1.A2019065.h24v05.006.2019083203309.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2019.03.14/MOD09A1.A2019073.h24v05.006.2019084023233.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2019.03.22/MOD09A1.A2019081.h24v05.006.2019091170526.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2019.03.30/MOD09A1.A2019089.h24v05.006.2019098145516.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2019.04.07/MOD09A1.A2019097.h24v05.006.2019106052023.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2019.04.15/MOD09A1.A2019105.h24v05.006.2019114035356.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2019.04.23/MOD09A1.A2019113.h24v05.006.2019122080425.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2019.05.01/MOD09A1.A2019121.h24v05.006.2019130034549.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2019.05.09/MOD09A1.A2019129.h24v05.006.2019150130259.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2019.05.17/MOD09A1.A2019137.h24v05.006.2019150090124.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2019.05.25/MOD09A1.A2019145.h24v05.006.2019154043128.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2019.06.02/MOD09A1.A2019153.h24v05.006.2019162042917.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2019.06.10/MOD09A1.A2019161.h24v05.006.2019170041600.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2019.06.18/MOD09A1.A2019169.h24v05.006.2019178035515.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2019.06.26/MOD09A1.A2019177.h24v05.006.2019186035209.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2019.07.04/MOD09A1.A2019185.h24v05.006.2019194035332.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2019.07.12/MOD09A1.A2019193.h24v05.006.2019202033454.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2019.07.20/MOD09A1.A2019201.h24v05.006.2019210042652.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2019.07.28/MOD09A1.A2019209.h24v05.006.2019218052503.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2019.08.05/MOD09A1.A2019217.h24v05.006.2019226052026.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2019.08.13/MOD09A1.A2019225.h24v05.006.2019235021333.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2019.08.21/MOD09A1.A2019233.h24v05.006.2019248170913.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2019.08.29/MOD09A1.A2019241.h24v05.006.2019250055533.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2019.09.06/MOD09A1.A2019249.h24v05.006.2019262204448.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2019.09.14/MOD09A1.A2019257.h24v05.006.2019269213234.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2019.09.22/MOD09A1.A2019265.h24v05.006.2019275210139.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2019.09.30/MOD09A1.A2019273.h24v05.006.2019283070050.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2019.10.08/MOD09A1.A2019281.h24v05.006.2019290040805.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2019.10.16/MOD09A1.A2019289.h24v05.006.2019304175834.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2019.10.24/MOD09A1.A2019297.h24v05.006.2019306041802.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2019.11.01/MOD09A1.A2019305.h24v05.006.2019317031739.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2019.11.09/MOD09A1.A2019313.h24v05.006.2019324003351.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2019.11.17/MOD09A1.A2019321.h24v05.006.2019330041535.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2019.11.25/MOD09A1.A2019329.h24v05.006.2019338045737.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2019.12.03/MOD09A1.A2019337.h24v05.006.2019346040238.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2019.12.11/MOD09A1.A2019345.h24v05.006.2019354184113.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2019.12.19/MOD09A1.A2019353.h24v05.006.2019362033006.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2019.12.27/MOD09A1.A2019361.h24v05.006.2020005030915.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2020.01.01/MOD09A1.A2020001.h24v05.006.2020010222531.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2020.01.09/MOD09A1.A2020009.h24v05.006.2020018035653.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2020.01.17/MOD09A1.A2020017.h24v05.006.2020028001603.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2020.01.25/MOD09A1.A2020025.h24v05.006.2020034193944.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2020.02.02/MOD09A1.A2020033.h24v05.006.2020042190427.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2020.02.10/MOD09A1.A2020041.h24v05.006.2020050071240.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2020.02.18/MOD09A1.A2020049.h24v05.006.2020058053940.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2020.02.26/MOD09A1.A2020057.h24v05.006.2020066043219.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2020.03.05/MOD09A1.A2020065.h24v05.006.2020074034030.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2020.03.13/MOD09A1.A2020073.h24v05.006.2020082044444.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2020.03.21/MOD09A1.A2020081.h24v05.006.2020090203405.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2020.03.29/MOD09A1.A2020089.h24v05.006.2020101013531.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2020.04.06/MOD09A1.A2020097.h24v05.006.2020106051943.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2020.04.14/MOD09A1.A2020105.h24v05.006.2020114044020.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2020.04.22/MOD09A1.A2020113.h24v05.006.2020122052826.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2020.04.30/MOD09A1.A2020121.h24v05.006.2020130055014.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2020.05.08/MOD09A1.A2020129.h24v05.006.2020139055019.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2020.05.16/MOD09A1.A2020137.h24v05.006.2020146033109.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2020.05.24/MOD09A1.A2020145.h24v05.006.2020154051837.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2020.06.01/MOD09A1.A2020153.h24v05.006.2020162053350.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2020.06.09/MOD09A1.A2020161.h24v05.006.2020170051131.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2020.06.17/MOD09A1.A2020169.h24v05.006.2020178042748.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2020.06.25/MOD09A1.A2020177.h24v05.006.2020188053515.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2020.07.03/MOD09A1.A2020185.h24v05.006.2020197032455.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2020.07.11/MOD09A1.A2020193.h24v05.006.2020202075352.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2020.07.19/MOD09A1.A2020201.h24v05.006.2020210051443.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2020.07.27/MOD09A1.A2020209.h24v05.006.2020218043822.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2020.08.04/MOD09A1.A2020217.h24v05.006.2020226041804.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2020.08.12/MOD09A1.A2020225.h24v05.006.2020234224748.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2020.08.20/MOD09A1.A2020233.h24v05.006.2020242031348.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2020.08.28/MOD09A1.A2020241.h24v05.006.2020281043212.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2020.09.05/MOD09A1.A2020249.h24v05.006.2020262020731.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2020.09.13/MOD09A1.A2020257.h24v05.006.2020267084845.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2020.09.21/MOD09A1.A2020265.h24v05.006.2020280043336.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2020.09.29/MOD09A1.A2020273.h24v05.006.2020290071442.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2020.10.07/MOD09A1.A2020281.h24v05.006.2020291114802.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2020.10.15/MOD09A1.A2020289.h24v05.006.2020301150203.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2020.10.23/MOD09A1.A2020297.h24v05.006.2020306041018.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2020.10.31/MOD09A1.A2020305.h24v05.006.2020315081940.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2020.11.08/MOD09A1.A2020313.h24v05.006.2020322035115.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2020.11.16/MOD09A1.A2020321.h24v05.006.2020338030558.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2020.11.24/MOD09A1.A2020329.h24v05.006.2020338104930.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2020.12.02/MOD09A1.A2020337.h24v05.006.2020347093311.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2020.12.10/MOD09A1.A2020345.h24v05.006.2020358220000.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2020.12.18/MOD09A1.A2020353.h24v05.006.2020364220941.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2020.12.26/MOD09A1.A2020361.h24v05.006.2021004035022.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2021.01.01/MOD09A1.A2021001.h24v05.006.2021010034502.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2021.01.09/MOD09A1.A2021009.h24v05.006.2021018032907.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2021.01.17/MOD09A1.A2021017.h24v05.006.2021026195000.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2021.01.25/MOD09A1.A2021025.h24v05.006.2021034040152.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2021.02.02/MOD09A1.A2021033.h24v05.006.2021042044534.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2021.02.10/MOD09A1.A2021041.h24v05.006.2021050032624.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2021.02.18/MOD09A1.A2021049.h24v05.006.2021061020328.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2021.02.26/MOD09A1.A2021057.h24v05.006.2021067224841.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2021.03.06/MOD09A1.A2021065.h24v05.006.2021075093352.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2021.03.14/MOD09A1.A2021073.h24v05.006.2021084160209.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2021.03.22/MOD09A1.A2021081.h24v05.006.2021090042748.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2021.03.30/MOD09A1.A2021089.h24v05.006.2021098043544.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2021.04.07/MOD09A1.A2021097.h24v05.006.2021107055012.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2021.04.15/MOD09A1.A2021105.h24v05.006.2021114050634.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2021.04.23/MOD09A1.A2021113.h24v05.006.2021122063034.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2021.05.01/MOD09A1.A2021121.h24v05.006.2021130034316.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2021.05.09/MOD09A1.A2021129.h24v05.006.2021138042801.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2021.05.17/MOD09A1.A2021137.h24v05.006.2021146082031.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2021.05.25/MOD09A1.A2021145.h24v05.006.2021154052557.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2021.06.02/MOD09A1.A2021153.h24v05.006.2021162051827.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2021.06.10/MOD09A1.A2021161.h24v05.006.2021170051437.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2021.06.18/MOD09A1.A2021169.h24v05.006.2021178054609.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2021.06.26/MOD09A1.A2021177.h24v05.006.2021188173017.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2021.07.04/MOD09A1.A2021185.h24v05.006.2021194062320.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2021.07.12/MOD09A1.A2021193.h24v05.006.2021202224841.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2021.07.20/MOD09A1.A2021201.h24v05.006.2021214021829.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2021.07.28/MOD09A1.A2021209.h24v05.006.2021221194042.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2021.08.05/MOD09A1.A2021217.h24v05.006.2021226061831.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2021.08.13/MOD09A1.A2021225.h24v05.006.2021234035912.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2021.08.21/MOD09A1.A2021233.h24v05.006.2021243154458.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2021.08.29/MOD09A1.A2021241.h24v05.006.2021251101657.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2021.09.06/MOD09A1.A2021249.h24v05.006.2021258054030.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2021.09.14/MOD09A1.A2021257.h24v05.006.2021266033914.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2021.09.22/MOD09A1.A2021265.h24v05.006.2021316132447.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2021.09.30/MOD09A1.A2021273.h24v05.006.2021282045524.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2021.10.08/MOD09A1.A2021281.h24v05.006.2021290045039.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2021.10.16/MOD09A1.A2021289.h24v05.006.2021299111124.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2021.10.24/MOD09A1.A2021297.h24v05.006.2021306212805.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2021.11.01/MOD09A1.A2021305.h24v05.006.2021314062931.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2021.11.09/MOD09A1.A2021313.h24v05.006.2021322051448.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2021.11.17/MOD09A1.A2021321.h24v05.006.2021331125119.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2021.11.25/MOD09A1.A2021329.h24v05.006.2021338212246.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2021.12.03/MOD09A1.A2021337.h24v05.006.2021346052532.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2021.12.11/MOD09A1.A2021345.h24v05.006.2021355054824.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2021.12.19/MOD09A1.A2021353.h24v05.006.2022003232344.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2021.12.27/MOD09A1.A2021361.h24v05.006.2022005043158.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2022.01.01/MOD09A1.A2022001.h24v05.006.2022010042423.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2022.01.09/MOD09A1.A2022009.h24v05.006.2022018215711.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2022.01.17/MOD09A1.A2022017.h24v05.006.2022026205313.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2022.01.25/MOD09A1.A2022025.h24v05.006.2022035022016.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2022.02.02/MOD09A1.A2022033.h24v05.006.2022042044151.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2022.02.10/MOD09A1.A2022041.h24v05.006.2022050141239.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2022.02.18/MOD09A1.A2022049.h24v05.006.2022058053025.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2022.02.26/MOD09A1.A2022057.h24v05.006.2022066034531.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2022.03.06/MOD09A1.A2022065.h24v05.006.2022074040831.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2022.03.14/MOD09A1.A2022073.h24v05.006.2022101231014.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2022.03.22/MOD09A1.A2022081.h24v05.006.2022097075309.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2022.03.30/MOD09A1.A2022089.h24v05.006.2022101184815.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2022.04.07/MOD09A1.A2022097.h24v05.006.2022120235906.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2022.04.15/MOD09A1.A2022105.h24v05.006.2022118021732.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2022.04.23/MOD09A1.A2022113.h24v05.006.2022122191420.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2022.05.01/MOD09A1.A2022121.h24v05.006.2022134043656.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2022.05.09/MOD09A1.A2022129.h24v05.006.2022141180403.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2022.05.17/MOD09A1.A2022137.h24v05.006.2022146041559.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2022.05.25/MOD09A1.A2022145.h24v05.006.2022158000236.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2022.06.02/MOD09A1.A2022153.h24v05.006.2022162060341.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2022.06.10/MOD09A1.A2022161.h24v05.006.2022170044136.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2022.06.18/MOD09A1.A2022169.h24v05.006.2022178040617.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2022.06.26/MOD09A1.A2022177.h24v05.006.2022186044135.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2022.07.04/MOD09A1.A2022185.h24v05.006.2022195055524.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2022.07.12/MOD09A1.A2022193.h24v05.006.2022202053217.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2022.07.20/MOD09A1.A2022201.h24v05.006.2022215023220.hdf
https://e4ftl01.cr.usgs.gov//DP109/MOLT/MOD09A1.006/2022.07.28/MOD09A1.A2022209.h24v05.006.2022218043640.hdf
EDSCEOF

Overwriting download.sh


In [13]:
!chmod 777 download.sh

In [14]:
!./download.sh

Enter your Earthdata Login or other provider supplied credentials

./download.sh: line 37: warning: command substitution: ignored null byte in input
./download.sh: line 38: [: !)!)!)VALUESArchiveMetadata.0Attr0.0��������������������������'/;GS[bcdefgh/MODAPSx/archive/f5661/ops7/running/AM1M_C6_64_L10m/36074079/MOD09A1.6.2000-049T00:00:00.000000Z.51024005.77442427.209251_1.hdfCDF0.0200: integer expression expected
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 6924k  100 6924k    0     0  84411      0  0:01:24  0:01:24 --:--:-- 2246k

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 73.2M  100 73.2M    0     0   734k      0  0:01:42  0:01:42 --:--:-- 22.1M

  % Total    % Received % Xferd  Average Speed   T

In [9]:
!wget --user='kroy0001' --password='/#j%kWrPA,8.HRe' -i download.sh

^C


# # DOWNLOADING MOD10_L2 daily

In [ ]:
!mkdir -p /content/drive/MyDrive/OUT/data/MOD10_L2v61daily
!ls /content/drive/MyDrive/OUT/
%cd /content/drive/MyDrive/OUT/data/MOD10_L2v61daily

In [ ]:
%cd /content/drive/MyDrive/OUT/data/MOD10_L2v61daily

just change >>>
```
time_start = '2018-03-01T00:00:00Z'
time_end = '2022-08-05T17:40:42Z'
bounding_box = '75.65,31,78.69,32.74'
```
as per requirement IF download gets interrupted then check in google drive the
```
MOD10A1.A2018108.h24v05.061.2021324032344.hdf
```
> `A2018108`  is "2018" year and "108" th day of the year i.e. "2018-04-18" << that means it have successfully downloaded tillthen now change 
>`time_start` to `2018-04-18T00:00:00Z `
>`time_end` to `2022-08-05T17:40:42Z`
>>even the bounding box is updatable to user required co-ordinates

In [ ]:

from __future__ import print_function
from google.colab import output

import base64
import getopt
import itertools
import json
import math
import netrc
import os.path
import ssl
import sys
import time
from getpass import getpass

try:
    from urllib.parse import urlparse
    from urllib.request import urlopen, Request, build_opener, HTTPCookieProcessor
    from urllib.error import HTTPError, URLError
except ImportError:
    from urlparse import urlparse
    from urllib2 import urlopen, Request, HTTPError, URLError, build_opener, HTTPCookieProcessor

short_name = 'MOD10_L2'
version = '61'
time_start = '2003-01-20T00:00:00Z'
time_end = '2022-08-11T09:21:39Z'
bounding_box = '75.67,31.13,77.84,32.59'
polygon = ''
filename_filter = ''
url_list = []

CMR_URL = 'https://cmr.earthdata.nasa.gov'
URS_URL = 'https://urs.earthdata.nasa.gov'
CMR_PAGE_SIZE = 2000
CMR_FILE_URL = ('{0}/search/granules.json?provider=NSIDC_ECS'
                '&sort_key[]=start_date&sort_key[]=producer_granule_id'
                '&scroll=true&page_size={1}'.format(CMR_URL, CMR_PAGE_SIZE))



def get_username():
    username = 'kroy0001'
    while not username:
      # For Python 2/3 compatibility:
      try:
          do_input = raw_input  # noqa
      except NameError:
          do_input = input
      username = do_input('Earthdata username (or press Return to use a bearer token): ')
    return username


def get_password():
    password = '/#j%kWrPA,8.HRe'
    while not password:
        password = getpass('password: ')
    return password


def get_token():
    token = ''
    while not token:
        token = getpass('bearer token: ')
    return token


def get_login_credentials():
    """Get user credentials from .netrc or prompt for input."""
    credentials = None
    token = None

    try:
        info = netrc.netrc()
        username, account, password = info.authenticators(urlparse(URS_URL).hostname)
        if username == 'token':
            token = password
        else:
            credentials = '{0}:{1}'.format(username, password)
            credentials = base64.b64encode(credentials.encode('ascii')).decode('ascii')
    except Exception:
        username = None
        password = None

    if not username:
        username = get_username()
        if len(username):
            password = get_password()
            credentials = '{0}:{1}'.format(username, password)
            credentials = base64.b64encode(credentials.encode('ascii')).decode('ascii')
        else:
            token = get_token()

    return credentials, token


def build_version_query_params(version):
    desired_pad_length = 3
    if len(version) > desired_pad_length:
        print('Version string too long: "{0}"'.format(version))
        quit()

    version = str(int(version))  # Strip off any leading zeros
    query_params = ''

    while len(version) <= desired_pad_length:
        padded_version = version.zfill(desired_pad_length)
        query_params += '&version={0}'.format(padded_version)
        desired_pad_length -= 1
    return query_params


def filter_add_wildcards(filter):
    if not filter.startswith('*'):
        filter = '*' + filter
    if not filter.endswith('*'):
        filter = filter + '*'
    return filter


def build_filename_filter(filename_filter):
    filters = filename_filter.split(',')
    result = '&options[producer_granule_id][pattern]=true'
    for filter in filters:
        result += '&producer_granule_id[]=' + filter_add_wildcards(filter)
    return result


def build_cmr_query_url(short_name, version, time_start, time_end,
                        bounding_box=None, polygon=None,
                        filename_filter=None):
    params = '&short_name={0}'.format(short_name)
    params += build_version_query_params(version)
    params += '&temporal[]={0},{1}'.format(time_start, time_end)
    if polygon:
        params += '&polygon={0}'.format(polygon)
    elif bounding_box:
        params += '&bounding_box={0}'.format(bounding_box)
    if filename_filter:
        params += build_filename_filter(filename_filter)
    return CMR_FILE_URL + params


def get_speed(time_elapsed, chunk_size):
    if time_elapsed <= 0:
        return ''
    speed = chunk_size / time_elapsed
    if speed <= 0:
        speed = 1
    size_name = ('', 'k', 'M', 'G', 'T', 'P', 'E', 'Z', 'Y')
    i = int(math.floor(math.log(speed, 1000)))
    p = math.pow(1000, i)
    return '{0:.1f}{1}B/s'.format(speed / p, size_name[i])


def output_progress(count, total, status='', bar_len=60):
    if total <= 0:
        return
    fraction = min(max(count / float(total), 0), 1)
    filled_len = int(round(bar_len * fraction))
    percents = int(round(100.0 * fraction))
    bar = '=' * filled_len + ' ' * (bar_len - filled_len)
    fmt = '  [{0}] {1:3d}%  {2}   '.format(bar, percents, status)
    print('\b' * (len(fmt) + 4), end='')  # clears the line
    sys.stdout.write(fmt)
    sys.stdout.flush()


def cmr_read_in_chunks(file_object, chunk_size=1024 * 1024):
    """Read a file in chunks using a generator. Default chunk size: 1Mb."""
    while True:
        data = file_object.read(chunk_size)
        if not data:
            break
        yield data


def get_login_response(url, credentials, token):
    opener = build_opener(HTTPCookieProcessor())

    req = Request(url)
    if token:
        req.add_header('Authorization', 'Bearer {0}'.format(token))
    elif credentials:
        try:
            response = opener.open(req)
            # We have a redirect URL - try again with authorization.
            url = response.url
        except HTTPError:
            # No redirect - just try again with authorization.
            pass
        except Exception as e:
            print('Error{0}: {1}'.format(type(e), str(e)))
            sys.exit(1)

        req = Request(url)
        req.add_header('Authorization', 'Basic {0}'.format(credentials))

    try:
        response = opener.open(req)
    except HTTPError as e:
        err = 'HTTP error {0}, {1}'.format(e.code, e.reason)
        if 'Unauthorized' in e.reason:
            if token:
                err += ': Check your bearer token'
            else:
                err += ': Check your username and password'
        print(err)
        sys.exit(1)
    except Exception as e:
        print('Error{0}: {1}'.format(type(e), str(e)))
        sys.exit(1)

    return response


def cmr_download(urls, force=False, quiet=False):
    """Download files from list of urls."""
    if not urls:
        return

    url_count = len(urls)
    if not quiet:
        print('Downloading {0} files...'.format(url_count))
    credentials = None
    token = None

    for index, url in enumerate(urls, start=1):
        if not credentials and not token:
            p = urlparse(url)
            if p.scheme == 'https':
                credentials, token = get_login_credentials()

        filename = url.split('/')[-1]
        if not quiet:
            print('{0}/{1}: {2}'.format(str(index).zfill(len(str(url_count))),
                                        url_count, filename))

        try:
            response = get_login_response(url, credentials, token)
            length = int(response.headers['content-length'])
            try:
                if not force and length == os.path.getsize(filename):
                    if not quiet:
                        print('  File exists, skipping')
                    continue
            except OSError:
                pass
            count = 0
            chunk_size = min(max(length, 1), 1024 * 1024)
            max_chunks = int(math.ceil(length / chunk_size))
            time_initial = time.time()
            with open(filename, 'wb') as out_file:
                for data in cmr_read_in_chunks(response, chunk_size=chunk_size):
                    out_file.write(data)
                    if not quiet:
                        count = count + 1
                        time_elapsed = time.time() - time_initial
                        download_speed = get_speed(time_elapsed, count * chunk_size)
                        output_progress(count, max_chunks, status=download_speed)
            if not quiet:
                print()
        except HTTPError as e:
            print('HTTP error {0}, {1}'.format(e.code, e.reason))
        except URLError as e:
            print('URL error: {0}'.format(e.reason))
        except IOError:
            raise


def cmr_filter_urls(search_results):
    """Select only the desired data files from CMR response."""
    if 'feed' not in search_results or 'entry' not in search_results['feed']:
        return []

    entries = [e['links']
               for e in search_results['feed']['entry']
               if 'links' in e]
    # Flatten "entries" to a simple list of links
    links = list(itertools.chain(*entries))

    urls = []
    unique_filenames = set()
    for link in links:
        if 'href' not in link:
            # Exclude links with nothing to download
            continue
        if 'inherited' in link and link['inherited'] is True:
            # Why are we excluding these links?
            continue
        if 'rel' in link and 'data#' not in link['rel']:
            # Exclude links which are not classified by CMR as "data" or "metadata"
            continue

        if 'title' in link and 'opendap' in link['title'].lower():
            # Exclude OPeNDAP links--they are responsible for many duplicates
            # This is a hack; when the metadata is updated to properly identify
            # non-datapool links, we should be able to do this in a non-hack way
            continue

        filename = link['href'].split('/')[-1]
        if filename in unique_filenames:
            # Exclude links with duplicate filenames (they would overwrite)
            continue
        unique_filenames.add(filename)

        urls.append(link['href'])

    return urls


def cmr_search(short_name, version, time_start, time_end,
               bounding_box='', polygon='', filename_filter='', quiet=False):
    """Perform a scrolling CMR query for files matching input criteria."""
    cmr_query_url = build_cmr_query_url(short_name=short_name, version=version,
                                        time_start=time_start, time_end=time_end,
                                        bounding_box=bounding_box,
                                        polygon=polygon, filename_filter=filename_filter)
    if not quiet:
        print('Querying for data:\n\t{0}\n'.format(cmr_query_url))

    cmr_scroll_id = None
    ctx = ssl.create_default_context()
    ctx.check_hostname = False
    ctx.verify_mode = ssl.CERT_NONE

    urls = []
    hits = 0
    while True:
        req = Request(cmr_query_url)
        if cmr_scroll_id:
            req.add_header('cmr-scroll-id', cmr_scroll_id)
        try:
            response = urlopen(req, context=ctx)
        except Exception as e:
            print('Error: ' + str(e))
            sys.exit(1)
        if not cmr_scroll_id:
            # Python 2 and 3 have different case for the http headers
            headers = {k.lower(): v for k, v in dict(response.info()).items()}
            cmr_scroll_id = headers['cmr-scroll-id']
            hits = int(headers['cmr-hits'])
            if not quiet:
                if hits > 0:
                    print('Found {0} matches.'.format(hits))
                else:
                    print('Found no matches.')
        search_page = response.read()
        search_page = json.loads(search_page.decode('utf-8'))
        url_scroll_results = cmr_filter_urls(search_page)
        if not url_scroll_results:
            break
        if not quiet and hits > CMR_PAGE_SIZE:
            print('.', end='')
            sys.stdout.flush()
        urls += url_scroll_results

    if not quiet and hits > CMR_PAGE_SIZE:
        print()
    return urls


def main(argv=None):
    global short_name, version, time_start, time_end, bounding_box, \
        polygon, filename_filter, url_list

    if argv is None:
        argv = sys.argv[1:]

    force = False
    quiet = False
    usage = 'usage: nsidc-download_***.py [--help, -h] [--force, -f] [--quiet, -q]'

    try:
        opts, args = getopt.getopt(argv, 'hfq', ['help', 'force', 'quiet'])
        for opt, _arg in opts:
            if opt in ('-f', '--force'):
                force = True
            elif opt in ('-q', '--quiet'):
                quiet = True
            elif opt in ('-h', '--help'):
                print(usage)
                sys.exit(0)
    except getopt.GetoptError as e:
        print(e.args[0])
        print(usage)
        sys.exit(1)

    # Supply some default search parameters, just for testing purposes.
    # These are only used if the parameters aren't filled in up above.
    if 'short_name' in short_name:
        short_name = 'ATL06'
        version = '003'
        time_start = '2018-10-14T00:00:00Z'
        time_end = '2021-01-08T21:48:13Z'
        bounding_box = ''
        polygon = ''
        filename_filter = '*ATL06_2020111121*'
        url_list = []
    output.clear()
    try:
        if not url_list:
            url_list = cmr_search(short_name, version, time_start, time_end,
                                  bounding_box=bounding_box, polygon=polygon,
                                  filename_filter=filename_filter, quiet=quiet)

        cmr_download(url_list, force=force, quiet=quiet)
    except KeyboardInterrupt:
        quit()


if __name__ == '__main__':
    main()


  [=========                                                   ]  15%  5.9MB/s     [==============                                              ]  23%  7.0MB/s     [==================                                          ]  31%  7.6MB/s     [=======================                                     ]  38%  7.9MB/s     [============================                                ]  46%  8.4MB/s     [================================                            ]  54%  8.6MB/s   

# # DOWNLOADING ATLAS/ICESat-2 L3A Land Ice Height, Version 5

In [ ]:
!mkdir -p /content/drive/MyDrive/OUT/data/ATL06
!ls /content/drive/MyDrive/OUT/
%cd /content/drive/MyDrive/OUT/data/ATL06

In [ ]:
!ls /content/drive/MyDrive/OUT/data

In [ ]:
%cd /content/drive/MyDrive/OUT/data/ATL06

just change >>>
```
time_start = '2018-03-01T00:00:00Z'
time_end = '2022-08-05T17:40:42Z'
bounding_box = '75.65,31,78.69,32.74'
```
as per requirement IF download gets interrupted then check in google drive the
```
MOD10A1.A2018108.h24v05.061.2021324032344.hdf
```
> `A2018108`  is "2018" year and "108" th day of the year i.e. "2018-04-18" << that means it have successfully downloaded tillthen now change 
>`time_start` to `2018-04-18T00:00:00Z `
>`time_end` to `2022-08-05T17:40:42Z`
>>even the bounding box is updatable to user required co-ordinates

In [ ]:

from __future__ import print_function

import base64
import getopt
import itertools
import json
import math
import netrc
import os.path
import ssl
import sys
import time
from getpass import getpass

try:
    from urllib.parse import urlparse
    from urllib.request import urlopen, Request, build_opener, HTTPCookieProcessor
    from urllib.error import HTTPError, URLError
except ImportError:
    from urlparse import urlparse
    from urllib2 import urlopen, Request, HTTPError, URLError, build_opener, HTTPCookieProcessor

short_name = 'ATL06'
version = '005'
time_start = '2018-10-14T00:00:00Z'
time_end = '2022-08-07T22:49:51Z'
bounding_box = '75.73,31.07,78.11,32.39'
polygon = ''
filename_filter = ''
url_list = []

CMR_URL = 'https://cmr.earthdata.nasa.gov'
URS_URL = 'https://urs.earthdata.nasa.gov'
CMR_PAGE_SIZE = 2000
CMR_FILE_URL = ('{0}/search/granules.json?provider=NSIDC_ECS'
                '&sort_key[]=start_date&sort_key[]=producer_granule_id'
                '&scroll=true&page_size={1}'.format(CMR_URL, CMR_PAGE_SIZE))


def get_username():
    username = 'kroy0001'
    while not username:
      # For Python 2/3 compatibility:
      try:
          do_input = raw_input  # noqa
      except NameError:
          do_input = input
      username = do_input('Earthdata username (or press Return to use a bearer token): ')
    return username


def get_password():
    password = '/#j%kWrPA,8.HRe'
    while not password:
        password = getpass('password: ')
    return password


def get_token():
    token = ''
    while not token:
        token = getpass('bearer token: ')
    return token


def get_login_credentials():
    """Get user credentials from .netrc or prompt for input."""
    credentials = None
    token = None

    try:
        info = netrc.netrc()
        username, account, password = info.authenticators(urlparse(URS_URL).hostname)
        if username == 'token':
            token = password
        else:
            credentials = '{0}:{1}'.format(username, password)
            credentials = base64.b64encode(credentials.encode('ascii')).decode('ascii')
    except Exception:
        username = None
        password = None

    if not username:
        username = get_username()
        if len(username):
            password = get_password()
            credentials = '{0}:{1}'.format(username, password)
            credentials = base64.b64encode(credentials.encode('ascii')).decode('ascii')
        else:
            token = get_token()

    return credentials, token


def build_version_query_params(version):
    desired_pad_length = 3
    if len(version) > desired_pad_length:
        print('Version string too long: "{0}"'.format(version))
        quit()

    version = str(int(version))  # Strip off any leading zeros
    query_params = ''

    while len(version) <= desired_pad_length:
        padded_version = version.zfill(desired_pad_length)
        query_params += '&version={0}'.format(padded_version)
        desired_pad_length -= 1
    return query_params


def filter_add_wildcards(filter):
    if not filter.startswith('*'):
        filter = '*' + filter
    if not filter.endswith('*'):
        filter = filter + '*'
    return filter


def build_filename_filter(filename_filter):
    filters = filename_filter.split(',')
    result = '&options[producer_granule_id][pattern]=true'
    for filter in filters:
        result += '&producer_granule_id[]=' + filter_add_wildcards(filter)
    return result


def build_cmr_query_url(short_name, version, time_start, time_end,
                        bounding_box=None, polygon=None,
                        filename_filter=None):
    params = '&short_name={0}'.format(short_name)
    params += build_version_query_params(version)
    params += '&temporal[]={0},{1}'.format(time_start, time_end)
    if polygon:
        params += '&polygon={0}'.format(polygon)
    elif bounding_box:
        params += '&bounding_box={0}'.format(bounding_box)
    if filename_filter:
        params += build_filename_filter(filename_filter)
    return CMR_FILE_URL + params


def get_speed(time_elapsed, chunk_size):
    if time_elapsed <= 0:
        return ''
    speed = chunk_size / time_elapsed
    if speed <= 0:
        speed = 1
    size_name = ('', 'k', 'M', 'G', 'T', 'P', 'E', 'Z', 'Y')
    i = int(math.floor(math.log(speed, 1000)))
    p = math.pow(1000, i)
    return '{0:.1f}{1}B/s'.format(speed / p, size_name[i])


def output_progress(count, total, status='', bar_len=60):
    if total <= 0:
        return
    fraction = min(max(count / float(total), 0), 1)
    filled_len = int(round(bar_len * fraction))
    percents = int(round(100.0 * fraction))
    bar = '=' * filled_len + ' ' * (bar_len - filled_len)
    fmt = '  [{0}] {1:3d}%  {2}   '.format(bar, percents, status)
    print('\b' * (len(fmt) + 4), end='')  # clears the line
    sys.stdout.write(fmt)
    sys.stdout.flush()


def cmr_read_in_chunks(file_object, chunk_size=1024 * 1024):
    """Read a file in chunks using a generator. Default chunk size: 1Mb."""
    while True:
        data = file_object.read(chunk_size)
        if not data:
            break
        yield data


def get_login_response(url, credentials, token):
    opener = build_opener(HTTPCookieProcessor())

    req = Request(url)
    if token:
        req.add_header('Authorization', 'Bearer {0}'.format(token))
    elif credentials:
        try:
            response = opener.open(req)
            # We have a redirect URL - try again with authorization.
            url = response.url
        except HTTPError:
            # No redirect - just try again with authorization.
            pass
        except Exception as e:
            print('Error{0}: {1}'.format(type(e), str(e)))
            sys.exit(1)

        req = Request(url)
        req.add_header('Authorization', 'Basic {0}'.format(credentials))

    try:
        response = opener.open(req)
    except HTTPError as e:
        err = 'HTTP error {0}, {1}'.format(e.code, e.reason)
        if 'Unauthorized' in e.reason:
            if token:
                err += ': Check your bearer token'
            else:
                err += ': Check your username and password'
        print(err)
        sys.exit(1)
    except Exception as e:
        print('Error{0}: {1}'.format(type(e), str(e)))
        sys.exit(1)

    return response


def cmr_download(urls, force=False, quiet=False):
    """Download files from list of urls."""
    if not urls:
        return

    url_count = len(urls)
    if not quiet:
        print('Downloading {0} files...'.format(url_count))
    credentials = None
    token = None

    for index, url in enumerate(urls, start=1):
        if not credentials and not token:
            p = urlparse(url)
            if p.scheme == 'https':
                credentials, token = get_login_credentials()

        filename = url.split('/')[-1]
        if not quiet:
            print('{0}/{1}: {2}'.format(str(index).zfill(len(str(url_count))),
                                        url_count, filename))

        try:
            response = get_login_response(url, credentials, token)
            length = int(response.headers['content-length'])
            try:
                if not force and length == os.path.getsize(filename):
                    if not quiet:
                        print('  File exists, skipping')
                    continue
            except OSError:
                pass
            count = 0
            chunk_size = min(max(length, 1), 1024 * 1024)
            max_chunks = int(math.ceil(length / chunk_size))
            time_initial = time.time()
            with open(filename, 'wb') as out_file:
                for data in cmr_read_in_chunks(response, chunk_size=chunk_size):
                    out_file.write(data)
                    if not quiet:
                        count = count + 1
                        time_elapsed = time.time() - time_initial
                        download_speed = get_speed(time_elapsed, count * chunk_size)
                        output_progress(count, max_chunks, status=download_speed)
            if not quiet:
                print()
        except HTTPError as e:
            print('HTTP error {0}, {1}'.format(e.code, e.reason))
        except URLError as e:
            print('URL error: {0}'.format(e.reason))
        except IOError:
            raise


def cmr_filter_urls(search_results):
    """Select only the desired data files from CMR response."""
    if 'feed' not in search_results or 'entry' not in search_results['feed']:
        return []

    entries = [e['links']
               for e in search_results['feed']['entry']
               if 'links' in e]
    # Flatten "entries" to a simple list of links
    links = list(itertools.chain(*entries))

    urls = []
    unique_filenames = set()
    for link in links:
        if 'href' not in link:
            # Exclude links with nothing to download
            continue
        if 'inherited' in link and link['inherited'] is True:
            # Why are we excluding these links?
            continue
        if 'rel' in link and 'data#' not in link['rel']:
            # Exclude links which are not classified by CMR as "data" or "metadata"
            continue

        if 'title' in link and 'opendap' in link['title'].lower():
            # Exclude OPeNDAP links--they are responsible for many duplicates
            # This is a hack; when the metadata is updated to properly identify
            # non-datapool links, we should be able to do this in a non-hack way
            continue

        filename = link['href'].split('/')[-1]
        if filename in unique_filenames:
            # Exclude links with duplicate filenames (they would overwrite)
            continue
        unique_filenames.add(filename)

        urls.append(link['href'])

    return urls


def cmr_search(short_name, version, time_start, time_end,
               bounding_box='', polygon='', filename_filter='', quiet=False):
    """Perform a scrolling CMR query for files matching input criteria."""
    cmr_query_url = build_cmr_query_url(short_name=short_name, version=version,
                                        time_start=time_start, time_end=time_end,
                                        bounding_box=bounding_box,
                                        polygon=polygon, filename_filter=filename_filter)
    if not quiet:
        print('Querying for data:\n\t{0}\n'.format(cmr_query_url))

    cmr_scroll_id = None
    ctx = ssl.create_default_context()
    ctx.check_hostname = False
    ctx.verify_mode = ssl.CERT_NONE

    urls = []
    hits = 0
    while True:
        req = Request(cmr_query_url)
        if cmr_scroll_id:
            req.add_header('cmr-scroll-id', cmr_scroll_id)
        try:
            response = urlopen(req, context=ctx)
        except Exception as e:
            print('Error: ' + str(e))
            sys.exit(1)
        if not cmr_scroll_id:
            # Python 2 and 3 have different case for the http headers
            headers = {k.lower(): v for k, v in dict(response.info()).items()}
            cmr_scroll_id = headers['cmr-scroll-id']
            hits = int(headers['cmr-hits'])
            if not quiet:
                if hits > 0:
                    print('Found {0} matches.'.format(hits))
                else:
                    print('Found no matches.')
        search_page = response.read()
        search_page = json.loads(search_page.decode('utf-8'))
        url_scroll_results = cmr_filter_urls(search_page)
        if not url_scroll_results:
            break
        if not quiet and hits > CMR_PAGE_SIZE:
            print('.', end='')
            sys.stdout.flush()
        urls += url_scroll_results

    if not quiet and hits > CMR_PAGE_SIZE:
        print()
    return urls


def main(argv=None):
    global short_name, version, time_start, time_end, bounding_box, \
        polygon, filename_filter, url_list

    if argv is None:
        argv = sys.argv[1:]

    force = False
    quiet = False
    usage = 'usage: nsidc-download_***.py [--help, -h] [--force, -f] [--quiet, -q]'

    try:
        opts, args = getopt.getopt(argv, 'hfq', ['help', 'force', 'quiet'])
        for opt, _arg in opts:
            if opt in ('-f', '--force'):
                force = True
            elif opt in ('-q', '--quiet'):
                quiet = True
            elif opt in ('-h', '--help'):
                print(usage)
                sys.exit(0)
    except getopt.GetoptError as e:
        print(e.args[0])
        print(usage)
        sys.exit(1)

    # Supply some default search parameters, just for testing purposes.
    # These are only used if the parameters aren't filled in up above.
    if 'short_name' in short_name:
        short_name = 'ATL06'
        version = '003'
        time_start = '2018-10-14T00:00:00Z'
        time_end = '2021-01-08T21:48:13Z'
        bounding_box = ''
        polygon = ''
        filename_filter = '*ATL06_2020111121*'
        url_list = []

    try:
        if not url_list:
            url_list = cmr_search(short_name, version, time_start, time_end,
                                  bounding_box=bounding_box, polygon=polygon,
                                  filename_filter=filename_filter, quiet=quiet)

        cmr_download(url_list, force=force, quiet=quiet)
    except KeyboardInterrupt:
        quit()


if __name__ == '__main__':
    main()


# # DOWNLOADING MOD10A.061 daily

In [ ]:
!mkdir -p /content/drive/MyDrive/OUT/data/MOD10A1v6daily
!ls /content/drive/MyDrive/OUT/
%cd /content/drive/MyDrive/OUT/data/MOD10A1v6daily

In [ ]:
%cd /content/drive/MyDrive/OUT/data/MOD10A1v6daily

just change >>>
```
time_start = '2018-03-01T00:00:00Z'
time_end = '2022-08-05T17:40:42Z'
bounding_box = '75.65,31,78.69,32.74'
```
as per requirement IF download gets interrupted then check in google drive the
```
MOD10A1.A2018108.h24v05.061.2021324032344.hdf
```
> `A2018108`  is "2018" year and "108" th day of the year i.e. "2018-04-18" << that means it have successfully downloaded tillthen now change 
>`time_start` to `2018-04-18T00:00:00Z `
>`time_end` to `2022-08-05T17:40:42Z`
>>even the bounding box is updatable to user required co-ordinates

In [ ]:

from __future__ import print_function

import base64
import getopt
import itertools
import json
import math
import netrc
import os.path
import ssl
import sys
import time
from getpass import getpass

try:
    from urllib.parse import urlparse
    from urllib.request import urlopen, Request, build_opener, HTTPCookieProcessor
    from urllib.error import HTTPError, URLError
except ImportError:
    from urlparse import urlparse
    from urllib2 import urlopen, Request, HTTPError, URLError, build_opener, HTTPCookieProcessor

short_name = 'MOD10A1'
version = '61'
time_start = '2015-05-26T00:00:00Z'
time_end = '2022-08-05T17:40:42Z'
bounding_box = '75.65,31,78.69,32.74'
polygon = ''
filename_filter = ''
url_list = []

CMR_URL = 'https://cmr.earthdata.nasa.gov'
URS_URL = 'https://urs.earthdata.nasa.gov'
CMR_PAGE_SIZE = 2000
CMR_FILE_URL = ('{0}/search/granules.json?provider=NSIDC_ECS'
                '&sort_key[]=start_date&sort_key[]=producer_granule_id'
                '&scroll=true&page_size={1}'.format(CMR_URL, CMR_PAGE_SIZE))


def get_username():
    username = 'kroy0001'
    while not username:
      # For Python 2/3 compatibility:
      try:
          do_input = raw_input  # noqa
      except NameError:
          do_input = input
      username = do_input('Earthdata username (or press Return to use a bearer token): ')
    return username


def get_password():
    password = '/#j%kWrPA,8.HRe'
    while not password:
        password = getpass('password: ')
    return password

def get_token():
    token = ''
    while not token:
        token = getpass('bearer token: ')
    return token


def get_login_credentials():
    """Get user credentials from .netrc or prompt for input."""
    credentials = None
    token = None

    try:
        info = netrc.netrc()
        username, account, password = info.authenticators(urlparse(URS_URL).hostname)
        if username == 'token':
            token = password
        else:
            credentials = '{0}:{1}'.format(username, password)
            credentials = base64.b64encode(credentials.encode('ascii')).decode('ascii')
    except Exception:
        username = None
        password = None

    if not username:
        username = get_username()
        if len(username):
            password = get_password()
            credentials = '{0}:{1}'.format(username, password)
            credentials = base64.b64encode(credentials.encode('ascii')).decode('ascii')
        else:
            token = get_token()

    return credentials, token


def build_version_query_params(version):
    desired_pad_length = 3
    if len(version) > desired_pad_length:
        print('Version string too long: "{0}"'.format(version))
        quit()

    version = str(int(version))  # Strip off any leading zeros
    query_params = ''

    while len(version) <= desired_pad_length:
        padded_version = version.zfill(desired_pad_length)
        query_params += '&version={0}'.format(padded_version)
        desired_pad_length -= 1
    return query_params


def filter_add_wildcards(filter):
    if not filter.startswith('*'):
        filter = '*' + filter
    if not filter.endswith('*'):
        filter = filter + '*'
    return filter


def build_filename_filter(filename_filter):
    filters = filename_filter.split(',')
    result = '&options[producer_granule_id][pattern]=true'
    for filter in filters:
        result += '&producer_granule_id[]=' + filter_add_wildcards(filter)
    return result


def build_cmr_query_url(short_name, version, time_start, time_end,
                        bounding_box=None, polygon=None,
                        filename_filter=None):
    params = '&short_name={0}'.format(short_name)
    params += build_version_query_params(version)
    params += '&temporal[]={0},{1}'.format(time_start, time_end)
    if polygon:
        params += '&polygon={0}'.format(polygon)
    elif bounding_box:
        params += '&bounding_box={0}'.format(bounding_box)
    if filename_filter:
        params += build_filename_filter(filename_filter)
    return CMR_FILE_URL + params


def get_speed(time_elapsed, chunk_size):
    if time_elapsed <= 0:
        return ''
    speed = chunk_size / time_elapsed
    if speed <= 0:
        speed = 1
    size_name = ('', 'k', 'M', 'G', 'T', 'P', 'E', 'Z', 'Y')
    i = int(math.floor(math.log(speed, 1000)))
    p = math.pow(1000, i)
    return '{0:.1f}{1}B/s'.format(speed / p, size_name[i])


def output_progress(count, total, status='', bar_len=60):
    if total <= 0:
        return
    fraction = min(max(count / float(total), 0), 1)
    filled_len = int(round(bar_len * fraction))
    percents = int(round(100.0 * fraction))
    bar = '=' * filled_len + ' ' * (bar_len - filled_len)
    fmt = '  [{0}] {1:3d}%  {2}   '.format(bar, percents, status)
    print('\b' * (len(fmt) + 4), end='')  # clears the line
    sys.stdout.write(fmt)
    sys.stdout.flush()


def cmr_read_in_chunks(file_object, chunk_size=1024 * 1024):
    """Read a file in chunks using a generator. Default chunk size: 1Mb."""
    while True:
        data = file_object.read(chunk_size)
        if not data:
            break
        yield data


def get_login_response(url, credentials, token):
    opener = build_opener(HTTPCookieProcessor())

    req = Request(url)
    if token:
        req.add_header('Authorization', 'Bearer {0}'.format(token))
    elif credentials:
        try:
            response = opener.open(req)
            # We have a redirect URL - try again with authorization.
            url = response.url
        except HTTPError:
            # No redirect - just try again with authorization.
            pass
        except Exception as e:
            print('Error{0}: {1}'.format(type(e), str(e)))
            sys.exit(1)

        req = Request(url)
        req.add_header('Authorization', 'Basic {0}'.format(credentials))

    try:
        response = opener.open(req)
    except HTTPError as e:
        err = 'HTTP error {0}, {1}'.format(e.code, e.reason)
        if 'Unauthorized' in e.reason:
            if token:
                err += ': Check your bearer token'
            else:
                err += ': Check your username and password'
        print(err)
        sys.exit(1)
    except Exception as e:
        print('Error{0}: {1}'.format(type(e), str(e)))
        sys.exit(1)

    return response


def cmr_download(urls, force=False, quiet=False):
    """Download files from list of urls."""
    if not urls:
        return

    url_count = len(urls)
    if not quiet:
        print('Downloading {0} files...'.format(url_count))
    credentials = None
    token = None

    for index, url in enumerate(urls, start=1):
        if not credentials and not token:
            p = urlparse(url)
            if p.scheme == 'https':
                credentials, token = get_login_credentials()

        filename = url.split('/')[-1]
        if not quiet:
            print('{0}/{1}: {2}'.format(str(index).zfill(len(str(url_count))),
                                        url_count, filename))

        try:
            response = get_login_response(url, credentials, token)
            length = int(response.headers['content-length'])
            try:
                if not force and length == os.path.getsize(filename):
                    if not quiet:
                        print('  File exists, skipping')
                    continue
            except OSError:
                pass
            count = 0
            chunk_size = min(max(length, 1), 1024 * 1024)
            max_chunks = int(math.ceil(length / chunk_size))
            time_initial = time.time()
            with open(filename, 'wb') as out_file:
                for data in cmr_read_in_chunks(response, chunk_size=chunk_size):
                    out_file.write(data)
                    if not quiet:
                        count = count + 1
                        time_elapsed = time.time() - time_initial
                        download_speed = get_speed(time_elapsed, count * chunk_size)
                        output_progress(count, max_chunks, status=download_speed)
            if not quiet:
                print()
        except HTTPError as e:
            print('HTTP error {0}, {1}'.format(e.code, e.reason))
        except URLError as e:
            print('URL error: {0}'.format(e.reason))
        except IOError:
            raise


def cmr_filter_urls(search_results):
    """Select only the desired data files from CMR response."""
    if 'feed' not in search_results or 'entry' not in search_results['feed']:
        return []

    entries = [e['links']
               for e in search_results['feed']['entry']
               if 'links' in e]
    # Flatten "entries" to a simple list of links
    links = list(itertools.chain(*entries))

    urls = []
    unique_filenames = set()
    for link in links:
        if 'href' not in link:
            # Exclude links with nothing to download
            continue
        if 'inherited' in link and link['inherited'] is True:
            # Why are we excluding these links?
            continue
        if 'rel' in link and 'data#' not in link['rel']:
            # Exclude links which are not classified by CMR as "data" or "metadata"
            continue

        if 'title' in link and 'opendap' in link['title'].lower():
            # Exclude OPeNDAP links--they are responsible for many duplicates
            # This is a hack; when the metadata is updated to properly identify
            # non-datapool links, we should be able to do this in a non-hack way
            continue

        filename = link['href'].split('/')[-1]
        if filename in unique_filenames:
            # Exclude links with duplicate filenames (they would overwrite)
            continue
        unique_filenames.add(filename)

        urls.append(link['href'])

    return urls


def cmr_search(short_name, version, time_start, time_end,
               bounding_box='', polygon='', filename_filter='', quiet=False):
    """Perform a scrolling CMR query for files matching input criteria."""
    cmr_query_url = build_cmr_query_url(short_name=short_name, version=version,
                                        time_start=time_start, time_end=time_end,
                                        bounding_box=bounding_box,
                                        polygon=polygon, filename_filter=filename_filter)
    if not quiet:
        print('Querying for data:\n\t{0}\n'.format(cmr_query_url))

    cmr_scroll_id = None
    ctx = ssl.create_default_context()
    ctx.check_hostname = False
    ctx.verify_mode = ssl.CERT_NONE

    urls = []
    hits = 0
    while True:
        req = Request(cmr_query_url)
        if cmr_scroll_id:
            req.add_header('cmr-scroll-id', cmr_scroll_id)
        try:
            response = urlopen(req, context=ctx)
        except Exception as e:
            print('Error: ' + str(e))
            sys.exit(1)
        if not cmr_scroll_id:
            # Python 2 and 3 have different case for the http headers
            headers = {k.lower(): v for k, v in dict(response.info()).items()}
            cmr_scroll_id = headers['cmr-scroll-id']
            hits = int(headers['cmr-hits'])
            if not quiet:
                if hits > 0:
                    print('Found {0} matches.'.format(hits))
                else:
                    print('Found no matches.')
        search_page = response.read()
        search_page = json.loads(search_page.decode('utf-8'))
        url_scroll_results = cmr_filter_urls(search_page)
        if not url_scroll_results:
            break
        if not quiet and hits > CMR_PAGE_SIZE:
            print('.', end='')
            sys.stdout.flush()
        urls += url_scroll_results

    if not quiet and hits > CMR_PAGE_SIZE:
        print()
    return urls


def main(argv=None):
    global short_name, version, time_start, time_end, bounding_box, \
        polygon, filename_filter, url_list

    if argv is None:
        argv = sys.argv[1:]

    force = False
    quiet = False
    usage = 'usage: nsidc-download_***.py [--help, -h] [--force, -f] [--quiet, -q]'

    try:
        opts, args = getopt.getopt(argv, 'hfq', ['help', 'force', 'quiet'])
        for opt, _arg in opts:
            if opt in ('-f', '--force'):
                force = True
            elif opt in ('-q', '--quiet'):
                quiet = True
            elif opt in ('-h', '--help'):
                print(usage)
                sys.exit(0)
    except getopt.GetoptError as e:
        print(e.args[0])
        print(usage)
        sys.exit(1)

    # Supply some default search parameters, just for testing purposes.
    # These are only used if the parameters aren't filled in up above.
    if 'short_name' in short_name:
        short_name = 'ATL06'
        version = '003'
        time_start = '2018-10-14T00:00:00Z'
        time_end = '2021-01-08T21:48:13Z'
        bounding_box = ''
        polygon = ''
        filename_filter = '*ATL06_2020111121*'
        url_list = []

    try:
        if not url_list:
            url_list = cmr_search(short_name, version, time_start, time_end,
                                  bounding_box=bounding_box, polygon=polygon,
                                  filename_filter=filename_filter, quiet=quiet)

        cmr_download(url_list, force=force, quiet=quiet)
    except KeyboardInterrupt:
        quit()


if __name__ == '__main__':
    main()


# DOWNLOADING MOD10A.061 8 days

In [ ]:
!mkdir -p /content/drive/MyDrive/OUT/data/MOD10A1v6
!ls /content/drive/MyDrive/OUT/
%cd /content/drive/MyDrive/OUT/data/MOD10A1v6

In [ ]:
%cd /content/drive/MyDrive/OUT/data/MOD10A1v6
%pwd

In [ ]:

from __future__ import print_function

import base64
import getopt
import itertools
import json
import math
import netrc
import os.path
import ssl
import sys
import time
from getpass import getpass

try:
    from urllib.parse import urlparse
    from urllib.request import urlopen, Request, build_opener, HTTPCookieProcessor
    from urllib.error import HTTPError, URLError
except ImportError:
    from urlparse import urlparse
    from urllib2 import urlopen, Request, HTTPError, URLError, build_opener, HTTPCookieProcessor

short_name = 'MOD10A2'
version = '61'
time_start = '2000-02-18T00:00:00Z'
time_end = '2022-08-04T20:12:39Z'
bounding_box = '75.67,31.13,77.84,32.59'
polygon = ''
filename_filter = ''
url_list = []

CMR_URL = 'https://cmr.earthdata.nasa.gov'
URS_URL = 'https://urs.earthdata.nasa.gov'
CMR_PAGE_SIZE = 2000
CMR_FILE_URL = ('{0}/search/granules.json?provider=NSIDC_ECS'
                '&sort_key[]=start_date&sort_key[]=producer_granule_id'
                '&scroll=true&page_size={1}'.format(CMR_URL, CMR_PAGE_SIZE))


def get_username():
    username = 'kroy0001'
    while not username:
      # For Python 2/3 compatibility:
      try:
          do_input = raw_input  # noqa
      except NameError:
          do_input = input
      username = do_input('Earthdata username (or press Return to use a bearer token): ')
    return username


def get_password():
    password = '/#j%kWrPA,8.HRe'
    while not password:
        password = getpass('password: ')
    return password


def get_token():
    token = ''
    while not token:
        token = getpass('bearer token: ')
    return token


def get_login_credentials():
    """Get user credentials from .netrc or prompt for input."""
    credentials = None
    token = None

    try:
        info = netrc.netrc()
        username, account, password = info.authenticators(urlparse(URS_URL).hostname)
        if username == 'token':
            token = password
        else:
            credentials = '{0}:{1}'.format(username, password)
            credentials = base64.b64encode(credentials.encode('ascii')).decode('ascii')
    except Exception:
        username = None
        password = None

    if not username:
        username = get_username()
        if len(username):
            password = get_password()
            credentials = '{0}:{1}'.format(username, password)
            credentials = base64.b64encode(credentials.encode('ascii')).decode('ascii')
        else:
            token = get_token()

    return credentials, token


def build_version_query_params(version):
    desired_pad_length = 3
    if len(version) > desired_pad_length:
        print('Version string too long: "{0}"'.format(version))
        quit()

    version = str(int(version))  # Strip off any leading zeros
    query_params = ''

    while len(version) <= desired_pad_length:
        padded_version = version.zfill(desired_pad_length)
        query_params += '&version={0}'.format(padded_version)
        desired_pad_length -= 1
    return query_params


def filter_add_wildcards(filter):
    if not filter.startswith('*'):
        filter = '*' + filter
    if not filter.endswith('*'):
        filter = filter + '*'
    return filter


def build_filename_filter(filename_filter):
    filters = filename_filter.split(',')
    result = '&options[producer_granule_id][pattern]=true'
    for filter in filters:
        result += '&producer_granule_id[]=' + filter_add_wildcards(filter)
    return result


def build_cmr_query_url(short_name, version, time_start, time_end,
                        bounding_box=None, polygon=None,
                        filename_filter=None):
    params = '&short_name={0}'.format(short_name)
    params += build_version_query_params(version)
    params += '&temporal[]={0},{1}'.format(time_start, time_end)
    if polygon:
        params += '&polygon={0}'.format(polygon)
    elif bounding_box:
        params += '&bounding_box={0}'.format(bounding_box)
    if filename_filter:
        params += build_filename_filter(filename_filter)
    return CMR_FILE_URL + params


def get_speed(time_elapsed, chunk_size):
    if time_elapsed <= 0:
        return ''
    speed = chunk_size / time_elapsed
    if speed <= 0:
        speed = 1
    size_name = ('', 'k', 'M', 'G', 'T', 'P', 'E', 'Z', 'Y')
    i = int(math.floor(math.log(speed, 1000)))
    p = math.pow(1000, i)
    return '{0:.1f}{1}B/s'.format(speed / p, size_name[i])


def output_progress(count, total, status='', bar_len=60):
    if total <= 0:
        return
    fraction = min(max(count / float(total), 0), 1)
    filled_len = int(round(bar_len * fraction))
    percents = int(round(100.0 * fraction))
    bar = '=' * filled_len + ' ' * (bar_len - filled_len)
    fmt = '  [{0}] {1:3d}%  {2}   '.format(bar, percents, status)
    print('\b' * (len(fmt) + 4), end='')  # clears the line
    sys.stdout.write(fmt)
    sys.stdout.flush()


def cmr_read_in_chunks(file_object, chunk_size=1024 * 1024):
    """Read a file in chunks using a generator. Default chunk size: 1Mb."""
    while True:
        data = file_object.read(chunk_size)
        if not data:
            break
        yield data


def get_login_response(url, credentials, token):
    opener = build_opener(HTTPCookieProcessor())

    req = Request(url)
    if token:
        req.add_header('Authorization', 'Bearer {0}'.format(token))
    elif credentials:
        try:
            response = opener.open(req)
            # We have a redirect URL - try again with authorization.
            url = response.url
        except HTTPError:
            # No redirect - just try again with authorization.
            pass
        except Exception as e:
            print('Error{0}: {1}'.format(type(e), str(e)))
            sys.exit(1)

        req = Request(url)
        req.add_header('Authorization', 'Basic {0}'.format(credentials))

    try:
        response = opener.open(req)
    except HTTPError as e:
        err = 'HTTP error {0}, {1}'.format(e.code, e.reason)
        if 'Unauthorized' in e.reason:
            if token:
                err += ': Check your bearer token'
            else:
                err += ': Check your username and password'
        print(err)
        sys.exit(1)
    except Exception as e:
        print('Error{0}: {1}'.format(type(e), str(e)))
        sys.exit(1)

    return response


def cmr_download(urls, force=False, quiet=False):
    """Download files from list of urls."""
    if not urls:
        return

    url_count = len(urls)
    if not quiet:
        print('Downloading {0} files...'.format(url_count))
    credentials = None
    token = None

    for index, url in enumerate(urls, start=1):
        if not credentials and not token:
            p = urlparse(url)
            if p.scheme == 'https':
                credentials, token = get_login_credentials()

        filename = url.split('/')[-1]
        if not quiet:
            print('{0}/{1}: {2}'.format(str(index).zfill(len(str(url_count))),
                                        url_count, filename))

        try:
            response = get_login_response(url, credentials, token)
            length = int(response.headers['content-length'])
            try:
                if not force and length == os.path.getsize(filename):
                    if not quiet:
                        print('  File exists, skipping')
                    continue
            except OSError:
                pass
            count = 0
            chunk_size = min(max(length, 1), 1024 * 1024)
            max_chunks = int(math.ceil(length / chunk_size))
            time_initial = time.time()
            with open(filename, 'wb') as out_file:
                for data in cmr_read_in_chunks(response, chunk_size=chunk_size):
                    out_file.write(data)
                    if not quiet:
                        count = count + 1
                        time_elapsed = time.time() - time_initial
                        download_speed = get_speed(time_elapsed, count * chunk_size)
                        output_progress(count, max_chunks, status=download_speed)
            if not quiet:
                print()
        except HTTPError as e:
            print('HTTP error {0}, {1}'.format(e.code, e.reason))
        except URLError as e:
            print('URL error: {0}'.format(e.reason))
        except IOError:
            raise


def cmr_filter_urls(search_results):
    """Select only the desired data files from CMR response."""
    if 'feed' not in search_results or 'entry' not in search_results['feed']:
        return []

    entries = [e['links']
               for e in search_results['feed']['entry']
               if 'links' in e]
    # Flatten "entries" to a simple list of links
    links = list(itertools.chain(*entries))

    urls = []
    unique_filenames = set()
    for link in links:
        if 'href' not in link:
            # Exclude links with nothing to download
            continue
        if 'inherited' in link and link['inherited'] is True:
            # Why are we excluding these links?
            continue
        if 'rel' in link and 'data#' not in link['rel']:
            # Exclude links which are not classified by CMR as "data" or "metadata"
            continue

        if 'title' in link and 'opendap' in link['title'].lower():
            # Exclude OPeNDAP links--they are responsible for many duplicates
            # This is a hack; when the metadata is updated to properly identify
            # non-datapool links, we should be able to do this in a non-hack way
            continue

        filename = link['href'].split('/')[-1]
        if filename in unique_filenames:
            # Exclude links with duplicate filenames (they would overwrite)
            continue
        unique_filenames.add(filename)

        urls.append(link['href'])

    return urls


def cmr_search(short_name, version, time_start, time_end,
               bounding_box='', polygon='', filename_filter='', quiet=False):
    """Perform a scrolling CMR query for files matching input criteria."""
    cmr_query_url = build_cmr_query_url(short_name=short_name, version=version,
                                        time_start=time_start, time_end=time_end,
                                        bounding_box=bounding_box,
                                        polygon=polygon, filename_filter=filename_filter)
    if not quiet:
        print('Querying for data:\n\t{0}\n'.format(cmr_query_url))

    cmr_scroll_id = None
    ctx = ssl.create_default_context()
    ctx.check_hostname = False
    ctx.verify_mode = ssl.CERT_NONE

    urls = []
    hits = 0
    while True:
        req = Request(cmr_query_url)
        if cmr_scroll_id:
            req.add_header('cmr-scroll-id', cmr_scroll_id)
        try:
            response = urlopen(req, context=ctx)
        except Exception as e:
            print('Error: ' + str(e))
            sys.exit(1)
        if not cmr_scroll_id:
            # Python 2 and 3 have different case for the http headers
            headers = {k.lower(): v for k, v in dict(response.info()).items()}
            cmr_scroll_id = headers['cmr-scroll-id']
            hits = int(headers['cmr-hits'])
            if not quiet:
                if hits > 0:
                    print('Found {0} matches.'.format(hits))
                else:
                    print('Found no matches.')
        search_page = response.read()
        search_page = json.loads(search_page.decode('utf-8'))
        url_scroll_results = cmr_filter_urls(search_page)
        if not url_scroll_results:
            break
        if not quiet and hits > CMR_PAGE_SIZE:
            print('.', end='')
            sys.stdout.flush()
        urls += url_scroll_results

    if not quiet and hits > CMR_PAGE_SIZE:
        print()
    return urls


def main(argv=None):
    global short_name, version, time_start, time_end, bounding_box, \
        polygon, filename_filter, url_list

    if argv is None:
        argv = sys.argv[1:]

    force = False
    quiet = False
    usage = 'usage: nsidc-download_***.py [--help, -h] [--force, -f] [--quiet, -q]'

    try:
        opts, args = getopt.getopt(argv, 'hfq', ['help', 'force', 'quiet'])
        for opt, _arg in opts:
            if opt in ('-f', '--force'):
                force = True
            elif opt in ('-q', '--quiet'):
                quiet = True
            elif opt in ('-h', '--help'):
                print(usage)
                sys.exit(0)
    except getopt.GetoptError as e:
        print(e.args[0])
        print(usage)
        sys.exit(1)

    # Supply some default search parameters, just for testing purposes.
    # These are only used if the parameters aren't filled in up above.
    if 'short_name' in short_name:
        short_name = 'ATL06'
        version = '003'
        time_start = '2018-10-14T00:00:00Z'
        time_end = '2021-01-08T21:48:13Z'
        bounding_box = ''
        polygon = ''
        filename_filter = '*ATL06_2020111121*'
        url_list = []

    try:
        if not url_list:
            url_list = cmr_search(short_name, version, time_start, time_end,
                                  bounding_box=bounding_box, polygon=polygon,
                                  filename_filter=filename_filter, quiet=quiet)

        cmr_download(url_list, force=force, quiet=quiet)
    except KeyboardInterrupt:
        quit()


if __name__ == '__main__':
    main()
#1284 seconds execution time

# VIEW FIES

In [ ]:
%cd /content

In [ ]:
# Import packages
import os
import warnings
!python -m pip install xarray geopandas earthpy


import matplotlib.pyplot as plt
import numpy.ma as ma
import xarray as xr
import xarray as rxr
from shapely.geometry import mapping, box
import geopandas as gpd
import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep

warnings.simplefilter('ignore')

# Get the MODIS data
et.data.get_data('cold-springs-modis-h4')

# This download contains the fire boundary
et.data.get_data('cold-springs-fire')

# Set working directory
os.chdir(os.path.join(et.io.HOME,
                      'earth-analytics',
                      'data'))

In [ ]:
!cp -r /content/drive/MyDrive/OUT/data/MOD10A1v6/MOD10A1.A2000055.h24v05.006.2016061160550.hdf /content

In [ ]:
modis_pre_path = os.path.join("/content/MOD10A1.A2000055.h24v05.006.2016061160550.hdf")
modis_pre_path

In [ ]:
import xarray as rxr
modis_pre = rxr.open_rasterio(modis_pre_path, masked=True)
type(modis_pre)

In [ ]:
import rasterio
dataset = rasterio.open('MOD10A1.A2000320.h24v05.006.2016069160102.hdf')
dataset.name


In [ ]:
rasterio.open()



In [ ]:
%cd /content
!gdalinfo "/content/MOD10A2.A2022201.h24v05.061.2022215035527.hdf"

In [ ]:
!gdal_translate HDF4_EOS:EOS_GRID:"/content/MOD10A2.A2022201.h24v05.061.2022215035527.hdf":MOD_Grid_Snow_500m:Eight_Day_Snow_Cover /content/MOD10A2.A2022201.h24v05.061.2022215035527.img

NEW PROCESS BY NASA https://colab.research.google.com/github/astg606/py_materials/blob/master/science_data_format/introduction_h5py.ipynb

In [ ]:
%cd /content
import os
import datetime as dt
import six
import numpy as np
import h5py

hdFileName = '/MOD10A1.A2000320.h24v05.006.2016069160102.hdf'
modeType   = 'r'
hdfid = h5py.File(hdFileName, modeType)
print(hdfid.visit(print))
dct = dict()
def get_data(name, obj, dct=dct):
    if isinstance(obj, h5py.Dataset):
        _name = name if name.startswith('/') else '/'+name
        dct[_name] = obj[()]
with h5py.File(hdFileName, mode='r') as hdfid:         
     hdfid.visit(get_data)
    

introduction to xarray for reading hdf5 files of modis dataset

In [ ]:
!apt-get install libproj-dev proj-data proj-bin
!apt-get install libgeos-dev
!pip install cython
!pip install cartopy
!pip install netCDF4
!pip install xarray==0.16.2

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pprint
import cartopy
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.shapereader as shapereader

In [ ]:
import netCDF4
import numpy as np
import pandas as pd
import xarray as xr
print(f"Version of Numpy:   {np.__version__}")
print(f"Version of Pandas:  {pd.__version__}")
print(f"Version of netCDF4: {netCDF4.__version__}")
print(f"Version of Xarray:  {xr.__version__}")

In [ ]:
import netCDF4 as nc4

In [ ]:
!ls /content/drive/MyDrive/OUT/data/MOD10A1v6

In [ ]:
!cp /content/drive/MyDrive/OUT/data/MOD10A1v6/MOD10A2.A2021193.h24v05.061.2021202220805.hdf /content

In [ ]:
%cd /content/


In [ ]:
hdf5_fname = '/MOD10A1.A2000320.h24v05.006.2016069160102.hdf'
ncf = nc4.Dataset(hdf5_fname, diskless=True, persist=False)
print(ncf)


PROCESS FROM `H5NETCDF`

In [ ]:
!python -m pip install numpy h5py h5netcdf

import h5netcdf
import numpy as np

with h5netcdf.File('/MOD10A1.A2000320.h24v05.006.2016069160102.hdf', 'w') as f:
    # set dimensions with a dictionary
    f.dimensions = {'x': 5}
    # and update them with a dict-like interface
    # f.dimensions['x'] = 5
    # f.dimensions.update({'x': 5})

    v = f.create_variable('hello', ('x',), float)
    v[:] = np.ones(5)

    # you don't need to create groups first
    # you also don't need to create dimensions first if you supply data
    # with the new variable
    v = f.create_variable('/grouped/data', ('y',), data=np.arange(10))

    # access and modify attributes with a dict-like interface
    v.attrs['foo'] = 'bar'

    # you can access variables and groups directly using a hierarchical
    # keys like h5py
    print(f['/grouped/data'])

    # add an unlimited dimension
    f.dimensions['z'] = None
    # explicitly resize a dimension and all variables using it
    f.resize_dimension('z', 3)

In [ ]:
dataset = h5netcdf.File('/content/MOD10A1.A2000320.h24v05.006.2016069160102.hdf')
dataset.name

In [ ]:
# This is just a data exploration step
modis_pre_qa = modis_pre[0]
modis_pre_qa

In [ ]:
!python -m pip install rasterio
import rasterio as rio
with rio.open(modis_pre_path) as groups:
    for name in groups.subdatasets:
        print(name)

# test download

In [ ]:
%%writefile .netrc
machine urs.earthdata.nasa.gov
login kroy0001
password /#j%kWrPA,8.HRe

In [ ]:
%%writefile DAACDataDownload.py
"""
---------------------------------------------------------------------------------------------------
 How to Access the LP DAAC Data Pool with Python
 The following Python code example demonstrates how to configure a connection to download data from
 an Earthdata Login enabled server, specifically the LP DAAC's Data Pool.
---------------------------------------------------------------------------------------------------
 Author: Cole Krehbiel
 Last Updated: 05/14/2020
---------------------------------------------------------------------------------------------------
"""
# Load necessary packages into Python
from subprocess import Popen
from getpass import getpass
from netrc import netrc
import argparse
import time
import os
import requests

# ----------------------------------USER-DEFINED VARIABLES--------------------------------------- #
# Set up command line arguments
parser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter)
parser.add_argument('-dir', '--directory', required=True, help='Specify directory to save files to')
parser.add_argument('-f', '--files', required=True, help='A single granule URL, or the location of csv or textfile containing granule URLs')
args = parser.parse_args()

saveDir = args.directory  # Set local directory to download to
files = args.files        # Define file(s) to download from the LP DAAC Data Pool
prompts = ['Enter NASA Earthdata Login Username \n(or create an account at urs.earthdata.nasa.gov): ',
           'Enter NASA Earthdata Login Password: ']

# ---------------------------------SET UP WORKSPACE---------------------------------------------- #
# Create a list of files to download based on input type of files above
if files.endswith('.txt') or files.endswith('.csv'):
    fileList = open(files, 'r').readlines()  # If input is textfile w file URLs
elif isinstance(files, str):
    fileList = [files]                       # If input is a single file

# Generalize download directory
if saveDir[-1] != '/' and saveDir[-1] != '\\':
    saveDir = saveDir.strip("'").strip('"') + os.sep
urs = 'urs.earthdata.nasa.gov'    # Address to call for authentication

# --------------------------------AUTHENTICATION CONFIGURATION----------------------------------- #
# Determine if netrc file exists, and if so, if it includes NASA Earthdata Login Credentials
try:
    netrcDir = os.path.expanduser("~/.netrc")
    netrc(netrcDir).authenticators(urs)[0]

# Below, create a netrc file and prompt user for NASA Earthdata Login Username and Password
except FileNotFoundError:
    homeDir = os.path.expanduser("~")
    Popen('touch {0}.netrc | chmod og-rw {0}.netrc | echo machine {1} >> {0}.netrc'.format(homeDir + os.sep, urs), shell=True)
    Popen('echo login {} >> {}.netrc'.format(getpass(prompt=prompts[0]), homeDir + os.sep), shell=True)
    Popen('echo password {} >> {}.netrc'.format(getpass(prompt=prompts[1]), homeDir + os.sep), shell=True)

# Determine OS and edit netrc file if it exists but is not set up for NASA Earthdata Login
except TypeError:
    homeDir = os.path.expanduser("~")
    Popen('echo machine {1} >> {0}.netrc'.format(homeDir + os.sep, urs), shell=True)
    Popen('echo login {} >> {}.netrc'.format(getpass(prompt=prompts[0]), homeDir + os.sep), shell=True)
    Popen('echo password {} >> {}.netrc'.format(getpass(prompt=prompts[1]), homeDir + os.sep), shell=True)

# Delay for up to 1 minute to allow user to submit username and password before continuing
tries = 0
while tries < 30:
    try:
        netrc(netrcDir).authenticators(urs)[2]
    except:
        time.sleep(2.0)
    tries += 1

# -----------------------------------------DOWNLOAD FILE(S)-------------------------------------- #
# Loop through and download all files to the directory specified above, and keeping same filenames
for f in fileList:
    if not os.path.exists(saveDir):
        os.makedirs(saveDir)
    saveName = os.path.join(saveDir, f.split('/')[-1].strip())

    # Create and submit request and download file
    with requests.get(f.strip(), verify=False, stream=True, auth=(netrc(netrcDir).authenticators(urs)[0], netrc(netrcDir).authenticators(urs)[2])) as response:
        if response.status_code != 200:
            print("{} not downloaded. Verify that your username and password are correct in {}".format(f.split('/')[-1].strip(), netrcDir))
        else:
            response.raw.decode_content = True
            content = response.raw
            with open(saveName, 'wb') as d:
                while True:
                    chunk = content.read(16 * 1024)
                    if not chunk:
                        break
                    d.write(chunk)
            print('Downloaded file: {}'.format(saveName))
    #time.sleep(1.0)

In [ ]:
%%writefile EarthdataLoginSetup.py
# -*- coding: utf-8 -*-
"""
---------------------------------------------------------------------------------------------------
 How to Set Up Direct Access the LP DAAC Data Pool with Python
 The following Python code will configure a netrc profile that will allow users to download data
 from an Earthdata Login enabled server. See README for additional information
---------------------------------------------------------------------------------------------------
 Author: Cole Krehbiel
 Last Updated: 11/20/2018
-------------------------------------------------------------------------------
"""
# Load necessary packages into Python
from netrc import netrc
from subprocess import Popen
from getpass import getpass
import os

# -----------------------------------AUTHENTICATION CONFIGURATION-------------------------------- #
urs = 'urs.earthdata.nasa.gov'    # Earthdata URL to call for authentication
prompts = ['Enter NASA Earthdata Login Username \n(or create an account at urs.earthdata.nasa.gov): ',
           'Enter NASA Earthdata Login Password: ']

# Determine if netrc file exists, and if so, if it includes NASA Earthdata Login Credentials
try:
    netrcDir = os.path.expanduser("~/.netrc")
    netrc(netrcDir).authenticators(urs)[0]

# Below, create a netrc file and prompt user for NASA Earthdata Login Username and Password
except FileNotFoundError:
    homeDir = os.path.expanduser("~")
    Popen('touch {0}.netrc | chmod og-rw {0}.netrc | echo machine {1} >> {0}.netrc'.format(homeDir + os.sep, urs), shell=True)
    Popen('echo login {} >> {}.netrc'.format(getpass(prompt=prompts[0]), homeDir + os.sep), shell=True)
    Popen('echo password {} >> {}.netrc'.format(getpass(prompt=prompts[1]), homeDir + os.sep), shell=True)

# Determine OS and edit netrc file if it exists but is not set up for NASA Earthdata Login
except TypeError:
    homeDir = os.path.expanduser("~")
    Popen('echo machine {1} >> {0}.netrc'.format(homeDir + os.sep, urs), shell=True)
    Popen('echo login {} >> {}.netrc'.format(getpass(prompt=prompts[0]), homeDir + os.sep), shell=True)
    Popen('echo password {} >> {}.netrc'.format(getpass(prompt=prompts[1]), homeDir + os.sep), shell=True)

In [ ]:
# login kroy0001
# password /#j%kWrPA,8.HRe

In [ ]:
!python EarthdataLoginSetup.py

In [ ]:
!python DAACDataDownload.py -dir /content/sample_data -f /content/beasBasin-download-list_2022.txt

In [ ]:
%cd /content

In [ ]:
!python -m pip install requests
!touch /content/.netrc
!echo "machine urs.earthdata.nasa.gov login kroy0001 password /#j%kWrPA,8.HRe" >> .netrc
!chmod 0600 .netrc
!touch .urs_cookies

In [ ]:
!wget --load-cookies ~/.urs_cookies --save-cookies ~/.urs_cookies --auth-no-challenge=on --keep-session-cookies -i /content/beasBasin-download-list_2022.txt


In [ ]:
  # Set the URL string to point to a specific data URL. Some generic examples are:
   #   https://servername/data/path/file
   #   https://servername/opendap/path/file[.format[?subset]]
   #   https://servername/daac-bin/OTF/HTTP_services.cgi?KEYWORD=value[&KEYWORD=value]
URL = "https://appeears.earthdatacloud.nasa.gov/api/bundle/76c0c3d1-eeb4-4661-a696-d71088db6a7d/d6b9e400-433d-4742-98c0-5515ecc4f342/MOD10A1.006_NDSI_Snow_Cover_Algorithm_Flags_QA_doy2021235_aid0001.tif"
   
   # Set the FILENAME string to the data file name, the LABEL keyword value, or any customized name. 
FILENAME = 'your_filename_string_goes_here'
   
import requests
result = requests.get(URL)
try:
   result.raise_for_status()
   f = open(FILENAME,'wb')
   f.write(result.content)
   f.close()
   print('contents of URL written to '+FILENAME)
except:
   print('requests.get() returned an error code '+str(result.status_code))

# DOWNLOADING ***`MOD10A1v6`***

In [ ]:
%cd /content/drive/MyDrive/OUT/data/MOD10A1v6
%pwd

In [ ]:
#!/usr/bin/env python
# ----------------------------------------------------------------------------
# NSIDC Data Download Script
#
# Copyright (c) 2022 Regents of the University of Colorado
# Permission is hereby granted, free of charge, to any person obtaining
# a copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
# The above copyright notice and this permission notice shall be included
# in all copies or substantial portions of the Software.
#
# Tested in Python 2.7 and Python 3.4, 3.6, 3.7, 3.8, 3.9
#
# To run the script at a Linux, macOS, or Cygwin command-line terminal:
#   $ python nsidc-data-download.py
#
# On Windows, open Start menu -> Run and type cmd. Then type:
#     python nsidc-data-download.py
#
# The script will first search Earthdata for all matching files.
# You will then be prompted for your Earthdata username/password
# and the script will download the matching files.
#
# If you wish, you may store your Earthdata username/password in a .netrc
# file in your $HOME directory and the script will automatically attempt to
# read this file. The .netrc file should have the following format:
#    machine urs.earthdata.nasa.gov login MYUSERNAME password MYPASSWORD
# where 'MYUSERNAME' and 'MYPASSWORD' are your Earthdata credentials.
#
# Instead of a username/password, you may use an Earthdata bearer token.
# To construct a bearer token, log into Earthdata and choose "Generate Token".
# To use the token, when the script prompts for your username,
# just press Return (Enter). You will then be prompted for your token.
# You can store your bearer token in the .netrc file in the following format:
#    machine urs.earthdata.nasa.gov login token password MYBEARERTOKEN
# where 'MYBEARERTOKEN' is your Earthdata bearer token.
#
from __future__ import print_function

import base64
import getopt
import itertools
import json
import math
import netrc
import os.path
import ssl
import sys
import time
from getpass import getpass

try:
    from urllib.parse import urlparse
    from urllib.request import urlopen, Request, build_opener, HTTPCookieProcessor
    from urllib.error import HTTPError, URLError
except ImportError:
    from urlparse import urlparse
    from urllib2 import urlopen, Request, HTTPError, URLError, build_opener, HTTPCookieProcessor

short_name = 'MOD10A1'
version = '6'
time_start = '2000-02-24T00:00:00Z'
time_end = '2022-08-01T23:49:17Z'
bounding_box = '75.93,31.19,78.27,32.42'
polygon = ''
filename_filter = ''
url_list = []

CMR_URL = 'https://cmr.earthdata.nasa.gov'
URS_URL = 'https://urs.earthdata.nasa.gov'
CMR_PAGE_SIZE = 2000
CMR_FILE_URL = ('{0}/search/granules.json?provider=NSIDC_ECS'
                '&sort_key[]=start_date&sort_key[]=producer_granule_id'
                '&scroll=true&page_size={1}'.format(CMR_URL, CMR_PAGE_SIZE))


def get_username():
    username = 'kroy0001'
    while not username:
    # For Python 2/3 compatibility:
      try:
          do_input = raw_input  # noqa
      except NameError:
          do_input = input
      username = do_input('Earthdata username (or press Return to use a bearer token): ')
    return username


def get_password():
    password = '/#j%kWrPA,8.HRe'
    while not password:
        password = getpass('password: ')
    return password


def get_token():
    token = ''
    while not token:
        token = getpass('bearer token: ')
    return token


def get_login_credentials():
    """Get user credentials from .netrc or prompt for input."""
    credentials = None
    token = None

    try:
        info = netrc.netrc()
        username, account, password = info.authenticators(urlparse(URS_URL).hostname)
        if username == 'token':
            token = password
        else:
            credentials = '{0}:{1}'.format(username, password)
            credentials = base64.b64encode(credentials.encode('ascii')).decode('ascii')
    except Exception:
        username = None
        password = None

    if not username:
        username = get_username()
        if len(username):
            password = get_password()
            credentials = '{0}:{1}'.format(username, password)
            credentials = base64.b64encode(credentials.encode('ascii')).decode('ascii')
        else:
            token = get_token()

    return credentials, token


def build_version_query_params(version):
    desired_pad_length = 3
    if len(version) > desired_pad_length:
        print('Version string too long: "{0}"'.format(version))
        quit()

    version = str(int(version))  # Strip off any leading zeros
    query_params = ''

    while len(version) <= desired_pad_length:
        padded_version = version.zfill(desired_pad_length)
        query_params += '&version={0}'.format(padded_version)
        desired_pad_length -= 1
    return query_params


def filter_add_wildcards(filter):
    if not filter.startswith('*'):
        filter = '*' + filter
    if not filter.endswith('*'):
        filter = filter + '*'
    return filter


def build_filename_filter(filename_filter):
    filters = filename_filter.split(',')
    result = '&options[producer_granule_id][pattern]=true'
    for filter in filters:
        result += '&producer_granule_id[]=' + filter_add_wildcards(filter)
    return result


def build_cmr_query_url(short_name, version, time_start, time_end,
                        bounding_box=None, polygon=None,
                        filename_filter=None):
    params = '&short_name={0}'.format(short_name)
    params += build_version_query_params(version)
    params += '&temporal[]={0},{1}'.format(time_start, time_end)
    if polygon:
        params += '&polygon={0}'.format(polygon)
    elif bounding_box:
        params += '&bounding_box={0}'.format(bounding_box)
    if filename_filter:
        params += build_filename_filter(filename_filter)
    return CMR_FILE_URL + params


def get_speed(time_elapsed, chunk_size):
    if time_elapsed <= 0:
        return ''
    speed = chunk_size / time_elapsed
    if speed <= 0:
        speed = 1
    size_name = ('', 'k', 'M', 'G', 'T', 'P', 'E', 'Z', 'Y')
    i = int(math.floor(math.log(speed, 1000)))
    p = math.pow(1000, i)
    return '{0:.1f}{1}B/s'.format(speed / p, size_name[i])


def output_progress(count, total, status='', bar_len=60):
    if total <= 0:
        return
    fraction = min(max(count / float(total), 0), 1)
    filled_len = int(round(bar_len * fraction))
    percents = int(round(100.0 * fraction))
    bar = '=' * filled_len + ' ' * (bar_len - filled_len)
    fmt = '  [{0}] {1:3d}%  {2}   '.format(bar, percents, status)
    print('\b' * (len(fmt) + 4), end='')  # clears the line
    sys.stdout.write(fmt)
    sys.stdout.flush()


def cmr_read_in_chunks(file_object, chunk_size=1024 * 1024):
    """Read a file in chunks using a generator. Default chunk size: 1Mb."""
    while True:
        data = file_object.read(chunk_size)
        if not data:
            break
        yield data


def get_login_response(url, credentials, token):
    opener = build_opener(HTTPCookieProcessor())

    req = Request(url)
    if token:
        req.add_header('Authorization', 'Bearer {0}'.format(token))
    elif credentials:
        try:
            response = opener.open(req)
            # We have a redirect URL - try again with authorization.
            url = response.url
        except HTTPError:
            # No redirect - just try again with authorization.
            pass
        except Exception as e:
            print('Error{0}: {1}'.format(type(e), str(e)))
            sys.exit(1)

        req = Request(url)
        req.add_header('Authorization', 'Basic {0}'.format(credentials))

    try:
        response = opener.open(req)
    except HTTPError as e:
        err = 'HTTP error {0}, {1}'.format(e.code, e.reason)
        if 'Unauthorized' in e.reason:
            if token:
                err += ': Check your bearer token'
            else:
                err += ': Check your username and password'
        print(err)
        sys.exit(1)
    except Exception as e:
        print('Error{0}: {1}'.format(type(e), str(e)))
        sys.exit(1)

    return response


def cmr_download(urls, force=False, quiet=False):
    """Download files from list of urls."""
    if not urls:
        return

    url_count = len(urls)
    if not quiet:
        print('Downloading {0} files...'.format(url_count))
    credentials = None
    token = None

    for index, url in enumerate(urls, start=1):
        if not credentials and not token:
            p = urlparse(url)
            if p.scheme == 'https':
                credentials, token = get_login_credentials()

        filename = url.split('/')[-1]
        if not quiet:
            print('{0}/{1}: {2}'.format(str(index).zfill(len(str(url_count))),
                                        url_count, filename))

        try:
            response = get_login_response(url, credentials, token)
            length = int(response.headers['content-length'])
            try:
                if not force and length == os.path.getsize(filename):
                    if not quiet:
                        print('  File exists, skipping')
                    continue
            except OSError:
                pass
            count = 0
            chunk_size = min(max(length, 1), 1024 * 1024)
            max_chunks = int(math.ceil(length / chunk_size))
            time_initial = time.time()
            with open(filename, 'wb') as out_file:
                for data in cmr_read_in_chunks(response, chunk_size=chunk_size):
                    out_file.write(data)
                    if not quiet:
                        count = count + 1
                        time_elapsed = time.time() - time_initial
                        download_speed = get_speed(time_elapsed, count * chunk_size)
                        output_progress(count, max_chunks, status=download_speed)
            if not quiet:
                print()
        except HTTPError as e:
            print('HTTP error {0}, {1}'.format(e.code, e.reason))
        except URLError as e:
            print('URL error: {0}'.format(e.reason))
        except IOError:
            raise


def cmr_filter_urls(search_results):
    """Select only the desired data files from CMR response."""
    if 'feed' not in search_results or 'entry' not in search_results['feed']:
        return []

    entries = [e['links']
               for e in search_results['feed']['entry']
               if 'links' in e]
    # Flatten "entries" to a simple list of links
    links = list(itertools.chain(*entries))

    urls = []
    unique_filenames = set()
    for link in links:
        if 'href' not in link:
            # Exclude links with nothing to download
            continue
        if 'inherited' in link and link['inherited'] is True:
            # Why are we excluding these links?
            continue
        if 'rel' in link and 'data#' not in link['rel']:
            # Exclude links which are not classified by CMR as "data" or "metadata"
            continue

        if 'title' in link and 'opendap' in link['title'].lower():
            # Exclude OPeNDAP links--they are responsible for many duplicates
            # This is a hack; when the metadata is updated to properly identify
            # non-datapool links, we should be able to do this in a non-hack way
            continue

        filename = link['href'].split('/')[-1]
        if filename in unique_filenames:
            # Exclude links with duplicate filenames (they would overwrite)
            continue
        unique_filenames.add(filename)

        urls.append(link['href'])

    return urls


def cmr_search(short_name, version, time_start, time_end,
               bounding_box='', polygon='', filename_filter='', quiet=False):
    """Perform a scrolling CMR query for files matching input criteria."""
    cmr_query_url = build_cmr_query_url(short_name=short_name, version=version,
                                        time_start=time_start, time_end=time_end,
                                        bounding_box=bounding_box,
                                        polygon=polygon, filename_filter=filename_filter)
    if not quiet:
        print('Querying for data:\n\t{0}\n'.format(cmr_query_url))

    cmr_scroll_id = None
    ctx = ssl.create_default_context()
    ctx.check_hostname = False
    ctx.verify_mode = ssl.CERT_NONE

    urls = []
    hits = 0
    while True:
        req = Request(cmr_query_url)
        if cmr_scroll_id:
            req.add_header('cmr-scroll-id', cmr_scroll_id)
        try:
            response = urlopen(req, context=ctx)
        except Exception as e:
            print('Error: ' + str(e))
            sys.exit(1)
        if not cmr_scroll_id:
            # Python 2 and 3 have different case for the http headers
            headers = {k.lower(): v for k, v in dict(response.info()).items()}
            cmr_scroll_id = headers['cmr-scroll-id']
            hits = int(headers['cmr-hits'])
            if not quiet:
                if hits > 0:
                    print('Found {0} matches.'.format(hits))
                else:
                    print('Found no matches.')
        search_page = response.read()
        search_page = json.loads(search_page.decode('utf-8'))
        url_scroll_results = cmr_filter_urls(search_page)
        if not url_scroll_results:
            break
        if not quiet and hits > CMR_PAGE_SIZE:
            print('.', end='')
            sys.stdout.flush()
        urls += url_scroll_results

    if not quiet and hits > CMR_PAGE_SIZE:
        print()
    return urls


def main(argv=None):
    global short_name, version, time_start, time_end, bounding_box, \
        polygon, filename_filter, url_list

    if argv is None:
        argv = sys.argv[1:]

    force = False
    quiet = False
    usage = 'usage: nsidc-download_***.py [--help, -h] [--force, -f] [--quiet, -q]'

    try:
        opts, args = getopt.getopt(argv, 'hfq', ['help', 'force', 'quiet'])
        for opt, _arg in opts:
            if opt in ('-f', '--force'):
                force = True
            elif opt in ('-q', '--quiet'):
                quiet = True
            elif opt in ('-h', '--help'):
                print(usage)
                sys.exit(0)
    except getopt.GetoptError as e:
        print(e.args[0])
        print(usage)
        sys.exit(1)

    # Supply some default search parameters, just for testing purposes.
    # These are only used if the parameters aren't filled in up above.
    if 'short_name' in short_name:
        short_name = 'ATL06'
        version = '003'
        time_start = '2018-10-14T00:00:00Z'
        time_end = '2021-01-08T21:48:13Z'
        bounding_box = ''
        polygon = ''
        filename_filter = '*ATL06_2020111121*'
        url_list = []

    try:
        if not url_list:
            url_list = cmr_search(short_name, version, time_start, time_end,
                                  bounding_box=bounding_box, polygon=polygon,
                                  filename_filter=filename_filter, quiet=quiet)

        cmr_download(url_list, force=force, quiet=quiet)
    except KeyboardInterrupt:
        quit()


if __name__ == '__main__':
    main()


# DOWNLOADING ***`VNP10v1`***

In [ ]:
!mkdir -r /content/drive/MyDrive/OUT/data/VNP10v1

In [ ]:
%cd /content/drive/MyDrive/OUT/data/VNP10v1
%pwd

In [ ]:
#!/usr/bin/env python
# ----------------------------------------------------------------------------
# NSIDC Data Download Script
#
# Copyright (c) 2022 Regents of the University of Colorado
# Permission is hereby granted, free of charge, to any person obtaining
# a copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
# The above copyright notice and this permission notice shall be included
# in all copies or substantial portions of the Software.
#
# Tested in Python 2.7 and Python 3.4, 3.6, 3.7, 3.8, 3.9
#
# To run the script at a Linux, macOS, or Cygwin command-line terminal:
#   $ python nsidc-data-download.py
#
# On Windows, open Start menu -> Run and type cmd. Then type:
#     python nsidc-data-download.py
#
# The script will first search Earthdata for all matching files.
# You will then be prompted for your Earthdata username/password
# and the script will download the matching files.
#
# If you wish, you may store your Earthdata username/password in a .netrc
# file in your $HOME directory and the script will automatically attempt to
# read this file. The .netrc file should have the following format:
#    machine urs.earthdata.nasa.gov login MYUSERNAME password MYPASSWORD
# where 'MYUSERNAME' and 'MYPASSWORD' are your Earthdata credentials.
#
# Instead of a username/password, you may use an Earthdata bearer token.
# To construct a bearer token, log into Earthdata and choose "Generate Token".
# To use the token, when the script prompts for your username,
# just press Return (Enter). You will then be prompted for your token.
# You can store your bearer token in the .netrc file in the following format:
#    machine urs.earthdata.nasa.gov login token password MYBEARERTOKEN
# where 'MYBEARERTOKEN' is your Earthdata bearer token.
#
from __future__ import print_function

import base64
import getopt
import itertools
import json
import math
import netrc
import os.path
import ssl
import sys
import time
from getpass import getpass

try:
    from urllib.parse import urlparse
    from urllib.request import urlopen, Request, build_opener, HTTPCookieProcessor
    from urllib.error import HTTPError, URLError
except ImportError:
    from urlparse import urlparse
    from urllib2 import urlopen, Request, HTTPError, URLError, build_opener, HTTPCookieProcessor

short_name = 'VNP10'
version = '1'
time_start = '2012-01-19T00:00:00Z'
time_end = '2022-08-02T00:16:19Z'
bounding_box = '75.97,31.17,78.26,32.41'
polygon = ''
filename_filter = ''
url_list = []

CMR_URL = 'https://cmr.earthdata.nasa.gov'
URS_URL = 'https://urs.earthdata.nasa.gov'
CMR_PAGE_SIZE = 2000
CMR_FILE_URL = ('{0}/search/granules.json?provider=NSIDC_ECS'
                '&sort_key[]=start_date&sort_key[]=producer_granule_id'
                '&scroll=true&page_size={1}'.format(CMR_URL, CMR_PAGE_SIZE))

def get_username():
    username = 'kroy0001'
    while not username:
    # For Python 2/3 compatibility:
      try:
          do_input = raw_input  # noqa
      except NameError:
          do_input = input
      username = do_input('Earthdata username (or press Return to use a bearer token): ')
    return username


def get_password():
    password = '/#j%kWrPA,8.HRe'
    while not password:
        password = getpass('password: ')
    return password


def get_token():
    token = ''
    while not token:
        token = getpass('bearer token: ')
    return token


def get_login_credentials():
    """Get user credentials from .netrc or prompt for input."""
    credentials = None
    token = None

    try:
        info = netrc.netrc()
        username, account, password = info.authenticators(urlparse(URS_URL).hostname)
        if username == 'token':
            token = password
        else:
            credentials = '{0}:{1}'.format(username, password)
            credentials = base64.b64encode(credentials.encode('ascii')).decode('ascii')
    except Exception:
        username = None
        password = None

    if not username:
        username = get_username()
        if len(username):
            password = get_password()
            credentials = '{0}:{1}'.format(username, password)
            credentials = base64.b64encode(credentials.encode('ascii')).decode('ascii')
        else:
            token = get_token()

    return credentials, token


def build_version_query_params(version):
    desired_pad_length = 3
    if len(version) > desired_pad_length:
        print('Version string too long: "{0}"'.format(version))
        quit()

    version = str(int(version))  # Strip off any leading zeros
    query_params = ''

    while len(version) <= desired_pad_length:
        padded_version = version.zfill(desired_pad_length)
        query_params += '&version={0}'.format(padded_version)
        desired_pad_length -= 1
    return query_params


def filter_add_wildcards(filter):
    if not filter.startswith('*'):
        filter = '*' + filter
    if not filter.endswith('*'):
        filter = filter + '*'
    return filter


def build_filename_filter(filename_filter):
    filters = filename_filter.split(',')
    result = '&options[producer_granule_id][pattern]=true'
    for filter in filters:
        result += '&producer_granule_id[]=' + filter_add_wildcards(filter)
    return result


def build_cmr_query_url(short_name, version, time_start, time_end,
                        bounding_box=None, polygon=None,
                        filename_filter=None):
    params = '&short_name={0}'.format(short_name)
    params += build_version_query_params(version)
    params += '&temporal[]={0},{1}'.format(time_start, time_end)
    if polygon:
        params += '&polygon={0}'.format(polygon)
    elif bounding_box:
        params += '&bounding_box={0}'.format(bounding_box)
    if filename_filter:
        params += build_filename_filter(filename_filter)
    return CMR_FILE_URL + params


def get_speed(time_elapsed, chunk_size):
    if time_elapsed <= 0:
        return ''
    speed = chunk_size / time_elapsed
    if speed <= 0:
        speed = 1
    size_name = ('', 'k', 'M', 'G', 'T', 'P', 'E', 'Z', 'Y')
    i = int(math.floor(math.log(speed, 1000)))
    p = math.pow(1000, i)
    return '{0:.1f}{1}B/s'.format(speed / p, size_name[i])


def output_progress(count, total, status='', bar_len=60):
    if total <= 0:
        return
    fraction = min(max(count / float(total), 0), 1)
    filled_len = int(round(bar_len * fraction))
    percents = int(round(100.0 * fraction))
    bar = '=' * filled_len + ' ' * (bar_len - filled_len)
    fmt = '  [{0}] {1:3d}%  {2}   '.format(bar, percents, status)
    print('\b' * (len(fmt) + 4), end='')  # clears the line
    sys.stdout.write(fmt)
    sys.stdout.flush()


def cmr_read_in_chunks(file_object, chunk_size=1024 * 1024):
    """Read a file in chunks using a generator. Default chunk size: 1Mb."""
    while True:
        data = file_object.read(chunk_size)
        if not data:
            break
        yield data


def get_login_response(url, credentials, token):
    opener = build_opener(HTTPCookieProcessor())

    req = Request(url)
    if token:
        req.add_header('Authorization', 'Bearer {0}'.format(token))
    elif credentials:
        try:
            response = opener.open(req)
            # We have a redirect URL - try again with authorization.
            url = response.url
        except HTTPError:
            # No redirect - just try again with authorization.
            pass
        except Exception as e:
            print('Error{0}: {1}'.format(type(e), str(e)))
            sys.exit(1)

        req = Request(url)
        req.add_header('Authorization', 'Basic {0}'.format(credentials))

    try:
        response = opener.open(req)
    except HTTPError as e:
        err = 'HTTP error {0}, {1}'.format(e.code, e.reason)
        if 'Unauthorized' in e.reason:
            if token:
                err += ': Check your bearer token'
            else:
                err += ': Check your username and password'
        print(err)
        sys.exit(1)
    except Exception as e:
        print('Error{0}: {1}'.format(type(e), str(e)))
        sys.exit(1)

    return response


def cmr_download(urls, force=False, quiet=False):
    """Download files from list of urls."""
    if not urls:
        return

    url_count = len(urls)
    if not quiet:
        print('Downloading {0} files...'.format(url_count))
    credentials = None
    token = None

    for index, url in enumerate(urls, start=1):
        if not credentials and not token:
            p = urlparse(url)
            if p.scheme == 'https':
                credentials, token = get_login_credentials()

        filename = url.split('/')[-1]
        if not quiet:
            print('{0}/{1}: {2}'.format(str(index).zfill(len(str(url_count))),
                                        url_count, filename))

        try:
            response = get_login_response(url, credentials, token)
            length = int(response.headers['content-length'])
            try:
                if not force and length == os.path.getsize(filename):
                    if not quiet:
                        print('  File exists, skipping')
                    continue
            except OSError:
                pass
            count = 0
            chunk_size = min(max(length, 1), 1024 * 1024)
            max_chunks = int(math.ceil(length / chunk_size))
            time_initial = time.time()
            with open(filename, 'wb') as out_file:
                for data in cmr_read_in_chunks(response, chunk_size=chunk_size):
                    out_file.write(data)
                    if not quiet:
                        count = count + 1
                        time_elapsed = time.time() - time_initial
                        download_speed = get_speed(time_elapsed, count * chunk_size)
                        output_progress(count, max_chunks, status=download_speed)
            if not quiet:
                print()
        except HTTPError as e:
            print('HTTP error {0}, {1}'.format(e.code, e.reason))
        except URLError as e:
            print('URL error: {0}'.format(e.reason))
        except IOError:
            raise


def cmr_filter_urls(search_results):
    """Select only the desired data files from CMR response."""
    if 'feed' not in search_results or 'entry' not in search_results['feed']:
        return []

    entries = [e['links']
               for e in search_results['feed']['entry']
               if 'links' in e]
    # Flatten "entries" to a simple list of links
    links = list(itertools.chain(*entries))

    urls = []
    unique_filenames = set()
    for link in links:
        if 'href' not in link:
            # Exclude links with nothing to download
            continue
        if 'inherited' in link and link['inherited'] is True:
            # Why are we excluding these links?
            continue
        if 'rel' in link and 'data#' not in link['rel']:
            # Exclude links which are not classified by CMR as "data" or "metadata"
            continue

        if 'title' in link and 'opendap' in link['title'].lower():
            # Exclude OPeNDAP links--they are responsible for many duplicates
            # This is a hack; when the metadata is updated to properly identify
            # non-datapool links, we should be able to do this in a non-hack way
            continue

        filename = link['href'].split('/')[-1]
        if filename in unique_filenames:
            # Exclude links with duplicate filenames (they would overwrite)
            continue
        unique_filenames.add(filename)

        urls.append(link['href'])

    return urls


def cmr_search(short_name, version, time_start, time_end,
               bounding_box='', polygon='', filename_filter='', quiet=False):
    """Perform a scrolling CMR query for files matching input criteria."""
    cmr_query_url = build_cmr_query_url(short_name=short_name, version=version,
                                        time_start=time_start, time_end=time_end,
                                        bounding_box=bounding_box,
                                        polygon=polygon, filename_filter=filename_filter)
    if not quiet:
        print('Querying for data:\n\t{0}\n'.format(cmr_query_url))

    cmr_scroll_id = None
    ctx = ssl.create_default_context()
    ctx.check_hostname = False
    ctx.verify_mode = ssl.CERT_NONE

    urls = []
    hits = 0
    while True:
        req = Request(cmr_query_url)
        if cmr_scroll_id:
            req.add_header('cmr-scroll-id', cmr_scroll_id)
        try:
            response = urlopen(req, context=ctx)
        except Exception as e:
            print('Error: ' + str(e))
            sys.exit(1)
        if not cmr_scroll_id:
            # Python 2 and 3 have different case for the http headers
            headers = {k.lower(): v for k, v in dict(response.info()).items()}
            cmr_scroll_id = headers['cmr-scroll-id']
            hits = int(headers['cmr-hits'])
            if not quiet:
                if hits > 0:
                    print('Found {0} matches.'.format(hits))
                else:
                    print('Found no matches.')
        search_page = response.read()
        search_page = json.loads(search_page.decode('utf-8'))
        url_scroll_results = cmr_filter_urls(search_page)
        if not url_scroll_results:
            break
        if not quiet and hits > CMR_PAGE_SIZE:
            print('.', end='')
            sys.stdout.flush()
        urls += url_scroll_results

    if not quiet and hits > CMR_PAGE_SIZE:
        print()
    return urls


def main(argv=None):
    global short_name, version, time_start, time_end, bounding_box, \
        polygon, filename_filter, url_list

    if argv is None:
        argv = sys.argv[1:]

    force = False
    quiet = False
    usage = 'usage: nsidc-download_***.py [--help, -h] [--force, -f] [--quiet, -q]'

    try:
        opts, args = getopt.getopt(argv, 'hfq', ['help', 'force', 'quiet'])
        for opt, _arg in opts:
            if opt in ('-f', '--force'):
                force = True
            elif opt in ('-q', '--quiet'):
                quiet = True
            elif opt in ('-h', '--help'):
                print(usage)
                sys.exit(0)
    except getopt.GetoptError as e:
        print(e.args[0])
        print(usage)
        sys.exit(1)

    # Supply some default search parameters, just for testing purposes.
    # These are only used if the parameters aren't filled in up above.
    if 'short_name' in short_name:
        short_name = 'ATL06'
        version = '003'
        time_start = '2018-10-14T00:00:00Z'
        time_end = '2021-01-08T21:48:13Z'
        bounding_box = ''
        polygon = ''
        filename_filter = '*ATL06_2020111121*'
        url_list = []

    try:
        if not url_list:
            url_list = cmr_search(short_name, version, time_start, time_end,
                                  bounding_box=bounding_box, polygon=polygon,
                                  filename_filter=filename_filter, quiet=quiet)

        cmr_download(url_list, force=force, quiet=quiet)
    except KeyboardInterrupt:
        quit()


if __name__ == '__main__':
    main()
